In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
import pandas as pd
import os
import glob
import numpy as np

<IPython.core.display.Javascript object>

In [3]:
import warnings

# Suppress specific FutureWarning
warnings.filterwarnings("ignore", category=FutureWarning, module="seaborn._oldcore")


<IPython.core.display.Javascript object>

In [4]:
import matplotlib.pyplot as plt
import seaborn as sns

<IPython.core.display.Javascript object>

In [5]:
from sklearn.preprocessing import StandardScaler

<IPython.core.display.Javascript object>

In [6]:
from collections import deque, defaultdict


<IPython.core.display.Javascript object>

In [7]:
def read_csv_files_grouped(csv_files, plant, header=[0, 1]):
    results = []
    for filepath in csv_files:
        df = pd.read_csv(filepath, header=header, low_memory=False)
        df["plant"] = plant  # Add the 'plant' column
        results.append(df)
    df = pd.concat(results, ignore_index=True)
    return df

<IPython.core.display.Javascript object>

In [8]:
def read_csv_files_full(csv_files, plant):
    results = []
    for filepath in csv_files:
        df = pd.read_csv(filepath, low_memory=False)
        df["plant"] = plant  # Add the 'plant' column
        results.append(df)
    df = pd.concat(results, ignore_index=True)
    return df

<IPython.core.display.Javascript object>

### Renaming the Columns

In [9]:
def preprocess_rename_columns(df):
    df.columns = pd.MultiIndex.from_tuples(
        list(
            {
                col: (col[0], "") if "Unnamed" in col[1] else col for col in df.columns
            }.values()
        )
    )
    return df

<IPython.core.display.Javascript object>

### Changing the order_of columns Columns

In [10]:
def preprocess_change_columns_order(df, column, pos):
    # Get the list of columns
    cols = df.columns.tolist()
    # Remove the specified column
    cols.remove(column)
    # Insert the column at the desired position
    cols.insert(pos, column)
    # Reorder the DataFrame columns
    return df[cols]

<IPython.core.display.Javascript object>

In [11]:
def read_csv_files_path(csv_files_path_dict, path, plant):
    csv_files = glob.glob(os.path.join(path, "*.csv"))
    csv_files_path_dict[plant] = csv_files
    return csv_files_path_dict

<IPython.core.display.Javascript object>

# Topological Analysis - NEW

## Definitions

In [12]:
# Function to compare two models strictly based on means only
def dominates(model1, model2):
    """Return True if model1 strictly dominates model2, False otherwise."""
    # Check if model1 is at least as good as model2 in all metrics
    at_least_as_good = (
        model1["RMSE_mean"] <= model2["RMSE_mean"]
        and model1["MAE_mean"] <= model2["MAE_mean"]
        and model1["MAPE_mean"] <= model2["MAPE_mean"]
        and model1["R2_mean"] >= model2["R2_mean"]
    )

    # Check if model1 is strictly better in at least one metric
    strictly_better_in_one = (
        model1["RMSE_mean"] < model2["RMSE_mean"]
        or model1["MAE_mean"] < model2["MAE_mean"]
        or model1["MAPE_mean"] < model2["MAPE_mean"]
        or model1["R2_mean"] > model2["R2_mean"]
    )

    # Model1 strictly dominates Model2 if it is at least as good in all metrics and
    # strictly better in at least one metric
    return at_least_as_good and strictly_better_in_one

<IPython.core.display.Javascript object>

In [13]:
def get_dominance_matrix_and_graph(df):
    n = len(df)
    dominance_matrix = np.zeros((n, n), dtype=bool)

    for i in range(n):
        for j in range(n):
            if i != j:
                dominance_matrix[i, j] = dominates(df.iloc[i], df.iloc[j])

    # Create the dominance graph
    dominance_graph = {i: [] for i in range(n)}
    for i in range(n):
        for j in range(n):
            if dominance_matrix[i, j]:
                dominance_graph[i].append(j)

    return dominance_matrix, dominance_graph

<IPython.core.display.Javascript object>

In [14]:
# Topological Sorting
def topological_sort(graph):
    """Perform topological sorting on the given graph."""
    in_degree = {u: 0 for u in graph}
    for u in graph:
        for v in graph[u]:
            in_degree[v] += 1

    queue = deque([u for u in graph if in_degree[u] == 0])
    topo_order = []

    while queue:
        u = queue.popleft()
        topo_order.append(u)

        for v in graph[u]:
            in_degree[v] -= 1
            if in_degree[v] == 0:
                queue.append(v)

    return topo_order

<IPython.core.display.Javascript object>

In [15]:
# Function to find the top models in each group
def find_top_models(group):
    max_net_dominance = group["Net_Dominance"].max()
    top_models = group[group["Net_Dominance"] == max_net_dominance]
    return top_models

<IPython.core.display.Javascript object>

### Dominance Analysis and SCPM

In [16]:
def compute_scpm(df):
    scaler = StandardScaler()
    standardized_metrics = scaler.fit_transform(
        df[["RMSE_mean", "MAE_mean", "MAPE_mean", "R2_mean"]]
    )

    # Creating a new dataframe with standardized values
    standardized_df = pd.DataFrame(
        standardized_metrics,
        columns=["RMSE_mean", "MAE_mean", "MAPE_mean", "R2_mean"],
    )

    # Summing all metrics and subtracting R2_mean
    standardized_df["Result"] = (
        standardized_df[["RMSE_mean", "MAE_mean", "MAPE_mean"]].sum(axis=1)
        - standardized_df["R2_mean"]
    )

    df["SCPM"] = standardized_df["Result"]
    return df

<IPython.core.display.Javascript object>

In [17]:
def make_dominance_analysis(df):
    # Compute dominance matrix and graph
    dominance_matrix, dominance_graph = get_dominance_matrix_and_graph(df)

    # Get the topological order
    topo_order = topological_sort(dominance_graph)
    df = df.copy()
    n = len(topo_order)

    # Identify Dominant Models
    dominant_models = [
        df["Model_bkp_2"].iloc[i]
        for i in range(len(df))
        if not np.any(dominance_matrix[:, i])
    ]

    # Identify Non-Dominant Models
    non_dominant_models = [
        df["Model_bkp_2"].iloc[i]
        for i in range(len(df))
        if np.any(dominance_matrix[:, i])
    ]

    # Identify Models that are both dominated and dominate at least one
    intermediate_models = [
        df["Model_bkp_2"].iloc[i]
        for i in range(len(df))
        if np.any(dominance_matrix[:, i]) and np.any(dominance_matrix[i, :])
    ]

    # Add count of times each model is dominated by others
    df["Dominated_Count"] = np.sum(dominance_matrix, axis=0)

    # Classify models
    df["Classification"] = "intermediate_model"  # default to intermediate_model
    df.loc[
        ~np.any(dominance_matrix, axis=1) & np.any(dominance_matrix, axis=0),
        "Classification",
    ] = "non_dominant_model"
    df.loc[~np.any(dominance_matrix, axis=0), "Classification"] = "dominant_model"

    # Map the topological order to the model names and get dominance counts
    sorted_models = [df["Model"].iloc[i] for i in topo_order]

    df_sorted = df.iloc[topo_order].copy()
    df_sorted["Dominates_Count"] = dominance_matrix.sum(axis=1)[topo_order]
    df_sorted_topo = df_sorted.copy()

    df_sorted = df_sorted.sort_values(by="Dominates_Count", ascending=False)
    df_sorted_count = df_sorted.copy()

    return {
        "dominance_matrix": dominance_matrix,
        "dominance_graph": dominance_graph,
        "df_sorted_topo": df_sorted_topo,
        "df_sorted_count": df_sorted_count,
    }

<IPython.core.display.Javascript object>

# Pre Training Analysis

# Reading The files

In [18]:
csv_files_path_fine_tuning = dict()
csv_files_path_pre_train = dict()

<IPython.core.display.Javascript object>

## 203

### Plant C

In [19]:
path_pre_train = "/home/peressim/projects/ccs28-ml-modelling/reports/results/global_models/203/c/pre_training/full/"
plant = "c"

csv_files_path_pre_train = read_csv_files_path(
    csv_files_path_pre_train, path_pre_train, plant
)

<IPython.core.display.Javascript object>

### Reading all data - pre training

#### Full

In [20]:
dfs_plant = dict()
column = ("plant", "")
pos = 2

for plant, csv_files in csv_files_path_pre_train.items():
    df = read_csv_files_full(csv_files, plant)
    df["plant"] = plant
    dfs_plant[plant] = df

<IPython.core.display.Javascript object>

In [21]:
df_pre_train = pd.concat(dfs_plant.values(), ignore_index=True)

<IPython.core.display.Javascript object>

In [22]:
df_pre_train.shape

(219, 23)

<IPython.core.display.Javascript object>

## Preprocessing

In [23]:
patterns = [
    "Chemical + Mineralogical + Feature Engineering",
    "Chemical + Mineralogical + CS7 + One-Hot",
    "Chemical + Mineralogical + Physical + One-Hot",
    "Chemical + Mineralogical + CS3 + One-Hot",
    "Chemical + Mineralogical + CS3 + CS7 + One-Hot",
    "Chemical + Mineralogical + One-Hot",
    "Chemical + Mineralogical + CS1 + CS3 + CS7 + One-Hot",
    "Chemical + Mineralogical + CS1 + CS3 + One-Hot",
    "Chemical + Mineralogical + CS1 + One-Hot",
    "Chemical + Mineralogical + CS2 + One-Hot",
    "Chemical + Feature Engineering",
]

FEATURES_TO_REPLACE_1 = {
    "Chemical + Mineralogical + CS3 + CS7": "Chemical + Mineralogical + Early CS",
    "Chemical + Mineralogical + CS7": "Chemical + Mineralogical + Early CS",
    "Chemical + Mineralogical + CS3": "Chemical + Mineralogical + Early CS",
    "Chemical + Mineralogical + CS1": "Chemical + Mineralogical + Early CS",
    "Chemical + Mineralogical + CS1 + CS3": "Chemical + Mineralogical + Early CS",
    "Chemical + Mineralogical + CS1 + CS3 + CS7": "Chemical + Mineralogical + Early CS",
    "Chemical + Mineralogical + CS2": "Chemical + Mineralogical + Early CS",
    "Chemical + CS1 + CS3 + CS7": "Chemical + Mineralogical + Early CS",
    "Chemical + CS3 + CS7": "Chemical + Mineralogical + Early CS",
    "Chemical + CS7": "Chemical + Mineralogical + Early CS",
    "Chemical + CS3": "Chemical + Mineralogical + Early CS",
    "Chemical + CS1 + CS3": "Chemical + Mineralogical + Early CS",
    "Chemical + CS1": "Chemical + Mineralogical + Early CS",
    "Chemical + CS1 + CS7": "Chemical + Mineralogical + Early CS",
}

FEATURES_TO_REPLACE_2 = {
    "Chemical": "Chemical + Mineralogical",
    "Chemical + Properties CS Less": "Chemical + Mineralogical + Physical - Early CS",
    "Chemical + Mineralogical + Properties CS Less": "Chemical + Mineralogical + Physical - Early CS",
    "Chemical + Physical": "Chemical + Mineralogical + Physical",
}

df_pre_train = df_pre_train[
    ~df_pre_train["Features"].apply(lambda x: x in patterns)
].reset_index(drop=True)

df_pre_train = df_pre_train[
    ~df_pre_train["Cross Validation"].eq("Out of time Split")
].reset_index(drop=True)

# removing afterwards
df_pre_train = df_pre_train[
    ~df_pre_train["Cross Validation"].eq("Repeated KFold")
].reset_index(drop=True)

df_pre_train["Features_bkp"] = df_pre_train["Features"].copy()
df_pre_train["Features"] = df_pre_train["Features"].replace(FEATURES_TO_REPLACE_1)

df_pre_train["Features"] = df_pre_train["Features"].replace(FEATURES_TO_REPLACE_2)


# removing afterwards
# df_pre_train = df_pre_train[
#     ~df_pre_train["Features"].eq("Chemical + Mineralogical + Early CS")
# ].reset_index(drop=True)

metrics_names = {
    "RMSE Test": "RMSE_mean",
    "MAE Test": "MAE_mean",
    "MAPE Test": "MAPE_mean",
    "R2 Test": "R2_mean",
}

df_pre_train = df_pre_train.rename(metrics_names, axis=1)

<IPython.core.display.Javascript object>

In [24]:
df_pre_train["Features"].unique()

array(['Chemical + Mineralogical + Physical - Early CS',
       'Chemical + Mineralogical + Physical', 'Chemical + Mineralogical'],
      dtype=object)

<IPython.core.display.Javascript object>

In [25]:
df_pre_train["MAPE Train"] = df_pre_train["MAPE Train"] * 100
df_pre_train["MAPE_mean"] = df_pre_train["MAPE_mean"] * 100

df_pre_train["Model_bkp"] = df_pre_train["Model"].copy()
df_pre_train["Model_bkp_2"] = df_pre_train["Model"].apply(
    lambda x: "" if pd.isna(x) else "_" + str(x)
)

<IPython.core.display.Javascript object>

In [26]:
df_pre_train["Model_bkp_2"] = df_pre_train["Model_bkp_2"].apply(
    lambda x: "MLP"
    if "MLP" in x
    else "Bi-LSTM"
    if "Bi-LSTM" in x
    else "LSTM"
    if "LSTM" in x
    else "Conv1D"
    if "Conv1D" in x
    else "Transformer"
)

<IPython.core.display.Javascript object>

In [27]:
df_pre_train["Cross Validation"].unique()

array(['Out of time'], dtype=object)

<IPython.core.display.Javascript object>

In [28]:
df_pre_train["Features"].unique()

array(['Chemical + Mineralogical + Physical - Early CS',
       'Chemical + Mineralogical + Physical', 'Chemical + Mineralogical'],
      dtype=object)

<IPython.core.display.Javascript object>

In [29]:
df_pre_train["Features_bkp"].unique()

array(['Chemical + Properties CS Less', 'Chemical + Physical', 'Chemical'],
      dtype=object)

<IPython.core.display.Javascript object>

In [30]:
df_pre_train.shape

(219, 26)

<IPython.core.display.Javascript object>

In [31]:
df_pre_train["Features"].unique()

array(['Chemical + Mineralogical + Physical - Early CS',
       'Chemical + Mineralogical + Physical', 'Chemical + Mineralogical'],
      dtype=object)

<IPython.core.display.Javascript object>

In [32]:
df_results_cm = (
    df_pre_train[df_pre_train["Features"].eq("Chemical + Mineralogical")]
    .copy()
    .reset_index(drop=True)
)

<IPython.core.display.Javascript object>

In [33]:
df_results_cm["Features"].unique()

array(['Chemical + Mineralogical'], dtype=object)

<IPython.core.display.Javascript object>

In [34]:
df_results_cm.shape

(73, 26)

<IPython.core.display.Javascript object>

# Global Analysis (pre train results)

## ECICS - 203

### Variable Grouping: CM

#### Analysis of the best models identified in the Local Analysis

**Best models for variable groupping CM were from:**

*BTiSS:*
    Dominance analysis: Plant S
    SCPM:Plant S

*TiSS:*
    Dominance analysis: Plant S
    SCPM:Plant S

#### Hold Out

In [35]:
df_results_cm = df_results_cm[df_results_cm["Plant"].eq("C")].reset_index(drop=True)

<IPython.core.display.Javascript object>

In [36]:
df_results_cm["Cross Validation"].unique()

array(['Out of time'], dtype=object)

<IPython.core.display.Javascript object>

In [37]:
df_results_cm_ho = (
    df_results_cm[df_results_cm["Cross Validation"].eq("Out of time")]
    .copy()
    .reset_index(drop=True)
)

<IPython.core.display.Javascript object>

In [38]:
df_results_cm_ho["Cross Validation"].unique()

array(['Out of time'], dtype=object)

<IPython.core.display.Javascript object>

In [39]:
df_results_cm_ho.shape

(73, 26)

<IPython.core.display.Javascript object>

###### SCPM computation

In [40]:
df_results_cm_ho = compute_scpm(df_results_cm_ho)

<IPython.core.display.Javascript object>

##### Dominance Analysis

In [41]:
dominance_dict = make_dominance_analysis(df_results_cm_ho)
dominance_matrix_cm_ho = dominance_dict["dominance_matrix"]
dominance_graph_cm_ho = dominance_dict["dominance_graph"]
df_sorted_count = dominance_dict["df_sorted_count"]
df_sorted_topo = dominance_dict["df_sorted_topo"]

<IPython.core.display.Javascript object>

In [42]:
dominance_matrix_cm_ho.shape, len(dominance_graph_cm_ho)

((73, 73), 73)

<IPython.core.display.Javascript object>

In [43]:
df_sorted_topo["Classification"].value_counts()

Classification
intermediate_model    71
dominant_model         1
non_dominant_model     1
Name: count, dtype: int64

<IPython.core.display.Javascript object>

In [44]:
# df_sorted_topo.to_csv(
#     "../../../../reports/results/local_models/ecics/ecics_dominance_analysis_cm_kf.csv",
#     index=False,
# )

<IPython.core.display.Javascript object>

In [45]:
df_sorted_topo.head(5)

,Category,Company,Plant,Features,Data Shape,Timesteps,Model,Model Params,Scaler,Scaler Params,...,MAPE_mean,R2_mean,plant,Features_bkp,Model_bkp,Model_bkp_2,SCPM,Dominated_Count,Classification,Dominates_Count
32,Global Model,203,C,Chemical + Mineralogical,"(64225, 10)",7.0,Bi-LSTM5,NaN,Standard Scaler,NaN,...,4.195080,0.841466,c,Chemical,Bi-LSTM5,Bi-LSTM,-0.985742,0,dominant_model,72
42,Global Model,203,C,Chemical + Mineralogical,"(64225, 10)",14.0,Bi-LSTM15,NaN,Standard Scaler,NaN,...,4.212489,0.838090,c,Chemical,Bi-LSTM15,Bi-LSTM,-0.966741,1,intermediate_model,71
35,Global Model,203,C,Chemical + Mineralogical,"(64225, 10)",7.0,Bi-LSTM8,NaN,Standard Scaler,NaN,...,4.309819,0.829291,c,Chemical,Bi-LSTM8,Bi-LSTM,-0.899827,2,intermediate_model,68
38,Global Model,203,C,Chemical + Mineralogical,"(64225, 10)",7.0,Bi-LSTM11,NaN,Standard Scaler,NaN,...,4.255023,0.825330,c,Chemical,Bi-LSTM11,Bi-LSTM,-0.897915,2,intermediate_model,67
17,Global Model,203,C,Chemical + Mineralogical,"(64225, 10)",7.0,LSTM5,NaN,Standard Scaler,NaN,...,4.341659,0.826918,c,Chemical,LSTM5,LSTM,-0.873286,3,intermediate_model,64


<IPython.core.display.Javascript object>

In [46]:
df_sorted_topo_cols = df_sorted_topo[
    [
        "Model",
        # "Model_bkp",
        # "Model_bkp_2",
        "Timesteps",
        "RMSE_mean",
        "MAE_mean",
        "MAPE_mean",
        "R2_mean",
        "SCPM",
        "Dominated_Count",
        "Classification",
        "Dominates_Count",
        "Plant",
    ]
].round(2)

<IPython.core.display.Javascript object>

##### Top dominant models

In [47]:
# Top 4
df_sorted_topo_cols.sort_values(by="Dominates_Count", ascending=False).iloc[0:3]

,Model,Timesteps,RMSE_mean,MAE_mean,MAPE_mean,R2_mean,SCPM,Dominated_Count,Classification,Dominates_Count,Plant
32,Bi-LSTM5,7.0,2.39,1.79,4.20,0.84,-0.99,0,dominant_model,72,C
42,Bi-LSTM15,14.0,2.42,1.80,4.21,0.84,-0.97,1,intermediate_model,71,C
35,Bi-LSTM8,7.0,2.48,1.85,4.31,0.83,-0.90,2,intermediate_model,68,C


<IPython.core.display.Javascript object>

##### Top intermediate models

In [48]:
df_sorted_topo_cols[
    df_sorted_topo_cols["Classification"].eq("intermediate_model")
].sort_values(by="Dominates_Count", ascending=False).iloc[:2]

,Model,Timesteps,RMSE_mean,MAE_mean,MAPE_mean,R2_mean,SCPM,Dominated_Count,Classification,Dominates_Count,Plant
42,Bi-LSTM15,14.0,2.42,1.80,4.21,0.84,-0.97,1,intermediate_model,71,C
35,Bi-LSTM8,7.0,2.48,1.85,4.31,0.83,-0.90,2,intermediate_model,68,C


<IPython.core.display.Javascript object>

##### Top non dominant models

In [49]:
df_sorted_topo_cols[
    df_sorted_topo_cols["Classification"].eq("non_dominant_model")
].sort_values(by="Dominated_Count", ascending=True).iloc[:2]

,Model,Timesteps,RMSE_mean,MAE_mean,MAPE_mean,R2_mean,SCPM,Dominated_Count,Classification,Dominates_Count,Plant
39,Bi-LSTM12,14.0,24.05,23.29,53.0,-15.03,33.78,72,non_dominant_model,0,C


<IPython.core.display.Javascript object>

In [50]:
df_sorted_topo_models_cols = df_sorted_topo[
    [
        # "Model",
        # "Model_bkp",
        "Model_bkp_2",
        "Timesteps",
        "RMSE_mean",
        "MAE_mean",
        "MAPE_mean",
        "R2_mean",
        "SCPM",
        "Dominated_Count",
        "Classification",
        "Dominates_Count",
        "Plant",
    ]
]  # .round(2)

<IPython.core.display.Javascript object>

In [51]:
df_sorted_topo["Model_bkp_2"].value_counts()

Model_bkp_2
Bi-LSTM        15
LSTM           15
Conv1D         15
Transformer    15
MLP            13
Name: count, dtype: int64

<IPython.core.display.Javascript object>

In [52]:
df_sorted_topo_models_grouped = (
    df_sorted_topo_models_cols.groupby("Model_bkp_2").describe().T
)

<IPython.core.display.Javascript object>

In [53]:
# pd.set_option("display.max_rows", None)
# df_sorted_topo_models_grouped

<IPython.core.display.Javascript object>

In [54]:
df_sorted_topo_models_cols = df_sorted_topo[
    [
        "Model_bkp_2",
        "Dominated_Count",
        "Dominates_Count",
        "Classification",
    ]
]

<IPython.core.display.Javascript object>

In [55]:
df_sorted_topo_models_cols.groupby("Model_bkp_2").describe().round(2)

Dominated_Count                                              \
                      count   mean    std   min   25%   50%   75%   max   
Model_bkp_2                                                               
Bi-LSTM                15.0  26.33  25.15   0.0   2.5  23.0  44.0  72.0   
Conv1D                 15.0  27.53  14.33   6.0  17.0  24.0  42.0  46.0   
LSTM                   15.0  26.93  21.14   3.0  11.5  15.0  50.5  60.0   
MLP                    13.0  22.54  11.07   3.0  13.0  25.0  31.0  38.0   
Transformer            15.0  57.80   9.14  43.0  49.0  60.0  66.5  69.0   

            Dominates_Count                                              
                      count   mean    std   min   25%   50%   75%   max  
Model_bkp_2                                                              
Bi-LSTM                15.0  41.67  25.81   0.0  23.5  41.0  65.0  72.0  
Conv1D                 15.0  35.80  13.83  18.0  23.0  38.0  46.5  56.0  
LSTM                   15.0  37.87  20.73   8.0  15.0  47.0  53.5  64.0  
MLP                    13.0  39.85   7.68  28.0  34.0  38.0  45.0  56.0  
Transformer            15.0   8.27   6.09   1.0   2.5   7.0  12.5  20.0

<IPython.core.display.Javascript object>

In [56]:
df_sorted_topo_models_cols.groupby("Model_bkp_2")["Classification"].describe()

,count,unique,top,freq
Model_bkp_2,,,,
Bi-LSTM,15,3,intermediate_model,13
Conv1D,15,1,intermediate_model,15
LSTM,15,1,intermediate_model,15
MLP,13,1,intermediate_model,13
Transformer,15,1,intermediate_model,15


<IPython.core.display.Javascript object>

In [57]:
df_sorted_topo_models_cols.groupby("Model_bkp_2")[["Classification"]].value_counts()

Model_bkp_2  Classification    
Bi-LSTM      intermediate_model    13
             dominant_model         1
             non_dominant_model     1
Conv1D       intermediate_model    15
LSTM         intermediate_model    15
MLP          intermediate_model    13
Transformer  intermediate_model    15
Name: count, dtype: int64

<IPython.core.display.Javascript object>

In [58]:
(
    df_sorted_topo_models_cols.groupby("Model_bkp_2")[["Classification"]]
    .value_counts(normalize=True)
    .to_frame()
    * 100
).round(2)

proportion
Model_bkp_2 Classification                
Bi-LSTM     intermediate_model       86.67
            dominant_model            6.67
            non_dominant_model        6.67
Conv1D      intermediate_model      100.00
LSTM        intermediate_model      100.00
MLP         intermediate_model      100.00
Transformer intermediate_model      100.00

<IPython.core.display.Javascript object>

In [59]:
df_sorted_topo_models_cols

,Model_bkp_2,Dominated_Count,Dominates_Count,Classification
32,Bi-LSTM,0,72,dominant_model
42,Bi-LSTM,1,71,intermediate_model
35,Bi-LSTM,2,68,intermediate_model
38,Bi-LSTM,2,67,intermediate_model
17,LSTM,3,64,intermediate_model
...,...,...,...,...
46,Transformer,67,1,intermediate_model
55,Transformer,67,1,intermediate_model
57,Transformer,67,2,intermediate_model
56,Transformer,69,1,intermediate_model


<IPython.core.display.Javascript object>

In [60]:
summary_stats = df_sorted_topo_models_cols.groupby("Model_bkp_2").agg(
    Total_Models=("Model_bkp_2", "count"),
    Mean_Dominance_Count=("Dominates_Count", "mean"),
    Median_Dominance_Count=("Dominates_Count", "median"),
    Total_Dominated=("Dominated_Count", "sum"),
    Total_Dominating=("Dominates_Count", "sum"),
)

total_models = summary_stats[
    "Total_Models"
].sum()  # Get the total number of models across all categories
summary_stats["Dominance Proportion"] = summary_stats["Total_Dominating"] / total_models

<IPython.core.display.Javascript object>

In [61]:
summary_stats

,Total_Models,Mean_Dominance_Count,Median_Dominance_Count,Total_Dominated,Total_Dominating,Dominance Proportion
Model_bkp_2,,,,,,
Bi-LSTM,15,41.666667,41.0,395,625,8.561644
Conv1D,15,35.800000,38.0,413,537,7.356164
LSTM,15,37.866667,47.0,404,568,7.780822
MLP,13,39.846154,38.0,293,518,7.095890
Transformer,15,8.266667,7.0,867,124,1.698630


<IPython.core.display.Javascript object>

In [62]:
df_sorted_topo_models_grouped.T

Timesteps                                                 \
                count      mean       std  min  25%  50%   75%   max   
Model_bkp_2                                                            
Bi-LSTM          15.0  7.333333  5.498918  1.0  1.0  7.0  14.0  14.0   
Conv1D           15.0  7.333333  5.498918  1.0  1.0  7.0  14.0  14.0   
LSTM             15.0  7.333333  5.498918  1.0  1.0  7.0  14.0  14.0   
MLP               0.0       NaN       NaN  NaN  NaN  NaN   NaN   NaN   
Transformer      15.0  7.333333  5.498918  1.0  1.0  7.0  14.0  14.0   

            RMSE_mean            ... Dominated_Count       Dominates_Count  \
                count      mean  ...             75%   max           count   
Model_bkp_2                      ...                                         
Bi-LSTM          15.0  4.153498  ...            44.0  72.0            15.0   
Conv1D           15.0  2.747037  ...            42.0  46.0            15.0   
LSTM             15.0  2.780299  ...            50.5  60.0            15.0   
MLP              13.0  2.749253  ...            31.0  38.0            13.0   
Transformer      15.0  3.240765  ...            66.5  69.0            15.0   

                                                                 
                  mean        std   min   25%   50%   75%   max  
Model_bkp_2                                                      
Bi-LSTM      41.666667  25.806053   0.0  23.5  41.0  65.0  72.0  
Conv1D       35.800000  13.826476  18.0  23.0  38.0  46.5  56.0  
LSTM         37.866667  20.732537   8.0  15.0  47.0  53.5  64.0  
MLP          39.846154   7.679476  28.0  34.0  38.0  45.0  56.0  
Transformer   8.266667   6.088240   1.0   2.5   7.0  12.5  20.0  

[5 rows x 64 columns]

<IPython.core.display.Javascript object>

### SCPM Analysis

In [63]:
cols = [
    "Model",
    # "Model_bkp",
    "Model_bkp_2",
    "Timesteps",
    "RMSE_mean",
    "MAE_mean",
    "MAPE_mean",
    "R2_mean",
    "SCPM",
    "Dominated_Count",
    "Dominates_Count",
    "Classification",
    "Plant",
]

<IPython.core.display.Javascript object>

In [64]:
df_sorted_scpm = df_sorted_topo.sort_values(by="SCPM").copy()

<IPython.core.display.Javascript object>

In [65]:
# (df_sorted_topo[cols].reset_index() == df_sorted_scpm[cols].reset_index()).all(axis=1)
df_sorted_scpm["topo_scmp_order_eq"] = df_sorted_topo.index == df_sorted_scpm.index

<IPython.core.display.Javascript object>

In [66]:
# df_sorted_topo[cols].round(2).head(5)

<IPython.core.display.Javascript object>

In [67]:
df_sorted_scpm[cols + ["topo_scmp_order_eq"]].round(2).head(7)

,Model,Model_bkp_2,Timesteps,RMSE_mean,MAE_mean,MAPE_mean,R2_mean,SCPM,Dominated_Count,Dominates_Count,Classification,Plant,topo_scmp_order_eq
32,Bi-LSTM5,Bi-LSTM,7.0,2.39,1.79,4.20,0.84,-0.99,0,72,dominant_model,C,True
42,Bi-LSTM15,Bi-LSTM,14.0,2.42,1.80,4.21,0.84,-0.97,1,71,intermediate_model,C,True
35,Bi-LSTM8,Bi-LSTM,7.0,2.48,1.85,4.31,0.83,-0.90,2,68,intermediate_model,C,True
38,Bi-LSTM11,Bi-LSTM,7.0,2.51,1.84,4.26,0.83,-0.90,2,67,intermediate_model,C,True
17,LSTM5,LSTM,7.0,2.50,1.88,4.34,0.83,-0.87,3,64,intermediate_model,C,True
41,Bi-LSTM14,Bi-LSTM,7.0,2.49,1.87,4.41,0.83,-0.87,3,63,intermediate_model,C,True
18,LSTM6,LSTM,14.0,2.51,1.87,4.37,0.83,-0.87,4,64,intermediate_model,C,False


<IPython.core.display.Javascript object>

In [68]:
df_sorted_scpm[cols + ["topo_scmp_order_eq"]].head(1)

,Model,Model_bkp_2,Timesteps,RMSE_mean,MAE_mean,MAPE_mean,R2_mean,SCPM,Dominated_Count,Dominates_Count,Classification,Plant,topo_scmp_order_eq
32,Bi-LSTM5,Bi-LSTM,7.0,2.390887,1.791395,4.19508,0.841466,-0.985742,0,72,dominant_model,C,True


<IPython.core.display.Javascript object>

### Variable Grouping: CM-P

#### Analysis of the best models identified in the Local Analysis

**Best models for variable groupping CM-P were from:**

*BTiSS:*
    Dominance analysis: Plant AM
    SCPM:Plant AM

*TiSS:*
    Dominance analysis: Plant W
    SCPM:Plant W

In [69]:
df_pre_train["Features"].unique()

array(['Chemical + Mineralogical + Physical - Early CS',
       'Chemical + Mineralogical + Physical', 'Chemical + Mineralogical'],
      dtype=object)

<IPython.core.display.Javascript object>

In [70]:
df_results_cm_p = (
    df_pre_train[
        df_pre_train["Features"].eq("Chemical + Mineralogical + Physical - Early CS")
    ]
    .copy()
    .reset_index(drop=True)
)

<IPython.core.display.Javascript object>

### Plant C

In [71]:
df_results_cm_p_am = df_results_cm_p[df_results_cm_p["Plant"].eq("C")].reset_index(
    drop=True
)

<IPython.core.display.Javascript object>

In [72]:
df_results_cm_p_am.shape

(73, 26)

<IPython.core.display.Javascript object>

#### Hold Out

In [73]:
df_results_cm_p_am["Cross Validation"].unique()

array(['Out of time'], dtype=object)

<IPython.core.display.Javascript object>

In [74]:
df_results_cm_p_am_ho = (
    df_results_cm_p_am[df_results_cm_p_am["Cross Validation"].eq("Out of time")]
    .copy()
    .reset_index(drop=True)
)

<IPython.core.display.Javascript object>

In [75]:
df_results_cm_p_am_ho["Cross Validation"].unique()

array(['Out of time'], dtype=object)

<IPython.core.display.Javascript object>

In [76]:
df_results_cm_p_am_ho.shape

(73, 26)

<IPython.core.display.Javascript object>

###### SCPM computation

In [77]:
df_results_cm_p_am_ho = compute_scpm(df_results_cm_p_am_ho)

<IPython.core.display.Javascript object>

##### Dominance Analysis

In [78]:
dominance_dict = make_dominance_analysis(df_results_cm_p_am_ho)
dominance_matrix_cm_p_am_ho = dominance_dict["dominance_matrix"]
dominance_graph_cm_p_am_ho = dominance_dict["dominance_graph"]
df_sorted_count = dominance_dict["df_sorted_count"]
df_sorted_topo = dominance_dict["df_sorted_topo"]

<IPython.core.display.Javascript object>

In [79]:
dominance_matrix_cm_p_am_ho.shape, len(dominance_graph_cm_p_am_ho)

((73, 73), 73)

<IPython.core.display.Javascript object>

In [80]:
df_sorted_topo["Classification"].value_counts()

Classification
intermediate_model    71
dominant_model         1
non_dominant_model     1
Name: count, dtype: int64

<IPython.core.display.Javascript object>

In [81]:
# df_sorted_topo.to_csv(
#     "../../../../reports/results/local_models/ecics/ecics_dominance_analysis_cm_kf.csv",
#     index=False,
# )

<IPython.core.display.Javascript object>

In [82]:
df_sorted_topo.head(5)

,Category,Company,Plant,Features,Data Shape,Timesteps,Model,Model Params,Scaler,Scaler Params,...,MAPE_mean,R2_mean,plant,Features_bkp,Model_bkp,Model_bkp_2,SCPM,Dominated_Count,Classification,Dominates_Count
66,Global Model,203,C,Chemical + Mineralogical + Physical - Early CS,"(64225, 15)",NaN,MLP_7,NaN,Standard Scaler,NaN,...,3.735687,0.868214,c,Chemical + Properties CS Less,MLP_7,MLP,-1.692946,0,dominant_model,72
65,Global Model,203,C,Chemical + Mineralogical + Physical - Early CS,"(64225, 15)",NaN,MLP_6,NaN,Standard Scaler,NaN,...,3.771993,0.867553,c,Chemical + Properties CS Less,MLP_6,MLP,-1.681048,1,intermediate_model,71
63,Global Model,203,C,Chemical + Mineralogical + Physical - Early CS,"(64225, 15)",NaN,MLP_4,NaN,Standard Scaler,NaN,...,3.792756,0.855092,c,Chemical + Properties CS Less,MLP_4,MLP,-1.629462,2,intermediate_model,64
69,Global Model,203,C,Chemical + Mineralogical + Physical - Early CS,"(64225, 15)",NaN,MLP_10,NaN,Standard Scaler,NaN,...,3.863786,0.857325,c,Chemical + Properties CS Less,MLP_10,MLP,-1.608431,2,intermediate_model,66
70,Global Model,203,C,Chemical + Mineralogical + Physical - Early CS,"(64225, 15)",NaN,MLP_11,NaN,Standard Scaler,NaN,...,3.874552,0.858174,c,Chemical + Properties CS Less,MLP_11,MLP,-1.616141,2,intermediate_model,66


<IPython.core.display.Javascript object>

In [83]:
df_sorted_topo_cols = df_sorted_topo[
    [
        "Model",
        # "Model_bkp",
        # "Model_bkp_2",
        "Timesteps",
        "RMSE_mean",
        "MAE_mean",
        "MAPE_mean",
        "R2_mean",
        "SCPM",
        "Dominated_Count",
        "Classification",
        "Dominates_Count",
        "Plant",
    ]
].round(2)

<IPython.core.display.Javascript object>

##### Top dominant models

In [84]:
# Top 4
df_sorted_topo_cols.sort_values(by="Dominates_Count", ascending=False).iloc[0:3]

,Model,Timesteps,RMSE_mean,MAE_mean,MAPE_mean,R2_mean,SCPM,Dominated_Count,Classification,Dominates_Count,Plant
66,MLP_7,NaN,2.18,1.58,3.74,0.87,-1.69,0,dominant_model,72,C
65,MLP_6,NaN,2.18,1.59,3.77,0.87,-1.68,1,intermediate_model,71,C
69,MLP_10,NaN,2.27,1.64,3.86,0.86,-1.61,2,intermediate_model,66,C


<IPython.core.display.Javascript object>

##### Top intermediate models

In [85]:
df_sorted_topo_cols[
    df_sorted_topo_cols["Classification"].eq("intermediate_model")
].sort_values(by="Dominates_Count", ascending=False).iloc[:2]

,Model,Timesteps,RMSE_mean,MAE_mean,MAPE_mean,R2_mean,SCPM,Dominated_Count,Classification,Dominates_Count,Plant
65,MLP_6,NaN,2.18,1.59,3.77,0.87,-1.68,1,intermediate_model,71,C
69,MLP_10,NaN,2.27,1.64,3.86,0.86,-1.61,2,intermediate_model,66,C


<IPython.core.display.Javascript object>

##### Top non dominant models

In [86]:
df_sorted_topo_cols[
    df_sorted_topo_cols["Classification"].eq("non_dominant_model")
].sort_values(by="Dominated_Count", ascending=True).iloc[:2]

,Model,Timesteps,RMSE_mean,MAE_mean,MAPE_mean,R2_mean,SCPM,Dominated_Count,Classification,Dominates_Count,Plant
41,Bi-LSTM12,14.0,24.38,23.63,53.8,-15.47,32.53,72,non_dominant_model,0,C


<IPython.core.display.Javascript object>

In [87]:
df_sorted_topo_models_cols = df_sorted_topo[
    [
        # "Model",
        # "Model_bkp",
        "Model_bkp_2",
        "Timesteps",
        "RMSE_mean",
        "MAE_mean",
        "MAPE_mean",
        "R2_mean",
        "SCPM",
        "Dominated_Count",
        "Classification",
        "Dominates_Count",
        "Plant",
    ]
]  # .round(2)

<IPython.core.display.Javascript object>

In [88]:
df_sorted_topo["Model_bkp_2"].value_counts()

Model_bkp_2
Conv1D         15
Bi-LSTM        15
LSTM           15
Transformer    15
MLP            13
Name: count, dtype: int64

<IPython.core.display.Javascript object>

In [89]:
df_sorted_topo_models_grouped = (
    df_sorted_topo_models_cols.groupby("Model_bkp_2").describe().T
)

<IPython.core.display.Javascript object>

In [90]:
# pd.set_option("display.max_rows", None)
# df_sorted_topo_models_grouped

<IPython.core.display.Javascript object>

In [91]:
df_sorted_topo_models_cols = df_sorted_topo[
    [
        "Model_bkp_2",
        "Dominated_Count",
        "Dominates_Count",
        "Classification",
    ]
]

<IPython.core.display.Javascript object>

In [92]:
df_sorted_topo_models_cols.groupby("Model_bkp_2").describe().round(2)

Dominated_Count                                              \
                      count   mean    std   min   25%   50%   75%   max   
Model_bkp_2                                                               
Bi-LSTM                15.0  52.20  16.30  13.0  45.0  55.0  64.5  72.0   
Conv1D                 15.0  22.47  10.92  12.0  16.5  20.0  25.5  58.0   
LSTM                   15.0  46.93  15.01  25.0  33.5  47.0  60.0  68.0   
MLP                    13.0   4.77   3.92   0.0   2.0   3.0   8.0  12.0   
Transformer            15.0  43.87  11.69  25.0  35.0  42.0  53.0  70.0   

            Dominates_Count                                              
                      count   mean    std   min   25%   50%   75%   max  
Model_bkp_2                                                              
Bi-LSTM                15.0  17.67  15.85   0.0   6.5  15.0  24.0  57.0  
Conv1D                 15.0  47.67  11.20  11.0  45.5  50.0  53.0  58.0  
LSTM                   15.0  22.60  14.33   4.0   9.5  22.0  34.0  46.0  
MLP                    13.0  64.69   3.75  59.0  62.0  65.0  66.0  72.0  
Transformer            15.0  25.60  11.17   1.0  18.0  26.0  33.0  46.0

<IPython.core.display.Javascript object>

In [93]:
df_sorted_topo_models_cols.groupby("Model_bkp_2")["Classification"].describe()

,count,unique,top,freq
Model_bkp_2,,,,
Bi-LSTM,15,2,intermediate_model,14
Conv1D,15,1,intermediate_model,15
LSTM,15,1,intermediate_model,15
MLP,13,2,intermediate_model,12
Transformer,15,1,intermediate_model,15


<IPython.core.display.Javascript object>

In [94]:
df_sorted_topo_models_cols.groupby("Model_bkp_2")[["Classification"]].value_counts()

Model_bkp_2  Classification    
Bi-LSTM      intermediate_model    14
             non_dominant_model     1
Conv1D       intermediate_model    15
LSTM         intermediate_model    15
MLP          intermediate_model    12
             dominant_model         1
Transformer  intermediate_model    15
Name: count, dtype: int64

<IPython.core.display.Javascript object>

In [95]:
(
    df_sorted_topo_models_cols.groupby("Model_bkp_2")[["Classification"]]
    .value_counts(normalize=True)
    .to_frame()
    * 100
).round(2)

proportion
Model_bkp_2 Classification                
Bi-LSTM     intermediate_model       93.33
            non_dominant_model        6.67
Conv1D      intermediate_model      100.00
LSTM        intermediate_model      100.00
MLP         intermediate_model       92.31
            dominant_model            7.69
Transformer intermediate_model      100.00

<IPython.core.display.Javascript object>

In [96]:
df_sorted_topo_models_cols

,Model_bkp_2,Dominated_Count,Dominates_Count,Classification
66,MLP,0,72,dominant_model
65,MLP,1,71,intermediate_model
63,MLP,2,64,intermediate_model
69,MLP,2,66,intermediate_model
70,MLP,2,66,intermediate_model
...,...,...,...,...
11,LSTM,68,4,intermediate_model
40,Bi-LSTM,69,3,intermediate_model
38,Bi-LSTM,70,1,intermediate_model
48,Transformer,70,1,intermediate_model


<IPython.core.display.Javascript object>

In [97]:
summary_stats = df_sorted_topo_models_cols.groupby("Model_bkp_2").agg(
    Total_Models=("Model_bkp_2", "count"),
    Mean_Dominance_Count=("Dominates_Count", "mean"),
    Median_Dominance_Count=("Dominates_Count", "median"),
    Total_Dominated=("Dominated_Count", "sum"),
    Total_Dominating=("Dominates_Count", "sum"),
)

total_models = summary_stats[
    "Total_Models"
].sum()  # Get the total number of models across all categories
summary_stats["Dominance Proportion"] = summary_stats["Total_Dominating"] / total_models

<IPython.core.display.Javascript object>

In [98]:
summary_stats

,Total_Models,Mean_Dominance_Count,Median_Dominance_Count,Total_Dominated,Total_Dominating,Dominance Proportion
Model_bkp_2,,,,,,
Bi-LSTM,15,17.666667,15.0,783,265,3.630137
Conv1D,15,47.666667,50.0,337,715,9.794521
LSTM,15,22.600000,22.0,704,339,4.643836
MLP,13,64.692308,65.0,62,841,11.520548
Transformer,15,25.600000,26.0,658,384,5.260274


<IPython.core.display.Javascript object>

In [99]:
df_sorted_topo_models_grouped.T

Timesteps                                                 \
                count      mean       std  min  25%  50%   75%   max   
Model_bkp_2                                                            
Bi-LSTM          15.0  7.333333  5.498918  1.0  1.0  7.0  14.0  14.0   
Conv1D           15.0  7.333333  5.498918  1.0  1.0  7.0  14.0  14.0   
LSTM             15.0  7.333333  5.498918  1.0  1.0  7.0  14.0  14.0   
MLP               0.0       NaN       NaN  NaN  NaN  NaN   NaN   NaN   
Transformer      15.0  7.333333  5.498918  1.0  1.0  7.0  14.0  14.0   

            RMSE_mean            ... Dominated_Count       Dominates_Count  \
                count      mean  ...             75%   max           count   
Model_bkp_2                      ...                                         
Bi-LSTM          15.0  5.314904  ...            64.5  72.0            15.0   
Conv1D           15.0  2.725186  ...            25.5  58.0            15.0   
LSTM             15.0  3.660430  ...            60.0  68.0            15.0   
MLP              13.0  2.277469  ...             8.0  12.0            13.0   
Transformer      15.0  3.449398  ...            53.0  70.0            15.0   

                                                                 
                  mean        std   min   25%   50%   75%   max  
Model_bkp_2                                                      
Bi-LSTM      17.666667  15.845985   0.0   6.5  15.0  24.0  57.0  
Conv1D       47.666667  11.197364  11.0  45.5  50.0  53.0  58.0  
LSTM         22.600000  14.326798   4.0   9.5  22.0  34.0  46.0  
MLP          64.692308   3.750214  59.0  62.0  65.0  66.0  72.0  
Transformer  25.600000  11.166276   1.0  18.0  26.0  33.0  46.0  

[5 rows x 64 columns]

<IPython.core.display.Javascript object>

### SCPM Analysis

In [100]:
cols = [
    "Model",
    # "Model_bkp",
    "Model_bkp_2",
    "Timesteps",
    "RMSE_mean",
    "MAE_mean",
    "MAPE_mean",
    "R2_mean",
    "SCPM",
    "Dominated_Count",
    "Dominates_Count",
    "Classification",
    "Plant",
]

<IPython.core.display.Javascript object>

In [101]:
df_sorted_scpm = df_sorted_topo.sort_values(by="SCPM").copy()

<IPython.core.display.Javascript object>

In [102]:
# (df_sorted_topo[cols].reset_index() == df_sorted_scpm[cols].reset_index()).all(axis=1)
df_sorted_scpm["topo_scmp_order_eq"] = df_sorted_topo.index == df_sorted_scpm.index

<IPython.core.display.Javascript object>

In [103]:
# df_sorted_topo[cols].round(2).head(5)

<IPython.core.display.Javascript object>

In [104]:
df_sorted_scpm[cols + ["topo_scmp_order_eq"]].round(2).head(7)

,Model,Model_bkp_2,Timesteps,RMSE_mean,MAE_mean,MAPE_mean,R2_mean,SCPM,Dominated_Count,Dominates_Count,Classification,Plant,topo_scmp_order_eq
66,MLP_7,MLP,NaN,2.18,1.58,3.74,0.87,-1.69,0,72,dominant_model,C,True
65,MLP_6,MLP,NaN,2.18,1.59,3.77,0.87,-1.68,1,71,intermediate_model,C,True
63,MLP_4,MLP,NaN,2.28,1.59,3.79,0.86,-1.63,2,64,intermediate_model,C,True
71,MLP_12,MLP,NaN,2.22,1.64,3.90,0.86,-1.62,2,66,intermediate_model,C,False
70,MLP_11,MLP,NaN,2.26,1.62,3.87,0.86,-1.62,2,66,intermediate_model,C,True
61,MLP_2,MLP,NaN,2.24,1.64,3.91,0.86,-1.61,3,65,intermediate_model,C,False
69,MLP_10,MLP,NaN,2.27,1.64,3.86,0.86,-1.61,2,66,intermediate_model,C,False


<IPython.core.display.Javascript object>

### Variable Grouping: CM-P-CS

#### Analysis of the best models identified in the Local Analysis

**Best models for variable groupping CM-P-CS were from:**

*BTiSS:*
    Dominance analysis: Plant S
    SCPM:Plant S and K

*TiSS:*
    Dominance analysis: Plant S
    SCPM:Plant S

In [105]:
df_pre_train["Features"].unique()

array(['Chemical + Mineralogical + Physical - Early CS',
       'Chemical + Mineralogical + Physical', 'Chemical + Mineralogical'],
      dtype=object)

<IPython.core.display.Javascript object>

In [106]:
df_results_cm_p_cs = (
    df_pre_train[df_pre_train["Features"].eq("Chemical + Mineralogical + Physical")]
    .copy()
    .reset_index(drop=True)
)

<IPython.core.display.Javascript object>

### Plant C

In [107]:
df_results_cm_p_cs_s = df_results_cm_p_cs[
    df_results_cm_p_cs["Plant"].eq("C")
].reset_index(drop=True)

<IPython.core.display.Javascript object>

In [108]:
df_results_cm_p_cs_s.shape

(73, 26)

<IPython.core.display.Javascript object>

#### Hold Out

In [109]:
df_results_cm_p_cs_s["Cross Validation"].unique()

array(['Out of time'], dtype=object)

<IPython.core.display.Javascript object>

In [110]:
df_results_cm_p_cs_s_ho = (
    df_results_cm_p_cs_s[df_results_cm_p_cs_s["Cross Validation"].eq("Out of time")]
    .copy()
    .reset_index(drop=True)
)

<IPython.core.display.Javascript object>

In [111]:
df_results_cm_p_cs_s_ho["Cross Validation"].unique()

array(['Out of time'], dtype=object)

<IPython.core.display.Javascript object>

In [112]:
df_results_cm_p_cs_s_ho.shape

(73, 26)

<IPython.core.display.Javascript object>

###### SCPM computation

In [113]:
df_results_cm_p_cs_s_ho = compute_scpm(df_results_cm_p_cs_s_ho)

<IPython.core.display.Javascript object>

##### Dominance Analysis

In [114]:
dominance_dict = make_dominance_analysis(df_results_cm_p_cs_s_ho)
dominance_matrix_cm_p_cs_s_ho = dominance_dict["dominance_matrix"]
dominance_graph_cm_p_cs_s_ho = dominance_dict["dominance_graph"]
df_sorted_count = dominance_dict["df_sorted_count"]
df_sorted_topo = dominance_dict["df_sorted_topo"]

<IPython.core.display.Javascript object>

In [115]:
dominance_matrix_cm_p_cs_s_ho.shape, len(dominance_graph_cm_p_cs_s_ho)

((73, 73), 73)

<IPython.core.display.Javascript object>

In [116]:
df_sorted_topo["Classification"].value_counts()

Classification
intermediate_model    71
dominant_model         1
non_dominant_model     1
Name: count, dtype: int64

<IPython.core.display.Javascript object>

In [117]:
# df_sorted_topo.to_csv(
#     "../../../../reports/results/local_models/ecics/ecics_dominance_analysis_cm_kf.csv",
#     index=False,
# )

<IPython.core.display.Javascript object>

In [118]:
df_sorted_topo.head(5)

,Category,Company,Plant,Features,Data Shape,Timesteps,Model,Model Params,Scaler,Scaler Params,...,MAPE_mean,R2_mean,plant,Features_bkp,Model_bkp,Model_bkp_2,SCPM,Dominated_Count,Classification,Dominates_Count
20,Global Model,203,C,Chemical + Mineralogical + Physical,"(64225, 17)",NaN,MLP_6,NaN,Standard Scaler,NaN,...,2.631175,0.936429,c,Chemical + Physical,MLP_6,MLP,-1.790568,0,dominant_model,72
22,Global Model,203,C,Chemical + Mineralogical + Physical,"(64225, 17)",NaN,MLP_8,NaN,Standard Scaler,NaN,...,2.676101,0.933405,c,Chemical + Physical,MLP_8,MLP,-1.762433,1,intermediate_model,67
24,Global Model,203,C,Chemical + Mineralogical + Physical,"(64225, 17)",NaN,MLP_10,NaN,Standard Scaler,NaN,...,2.674896,0.933494,c,Chemical + Physical,MLP_10,MLP,-1.762391,1,intermediate_model,67
26,Global Model,203,C,Chemical + Mineralogical + Physical,"(64225, 17)",NaN,MLP_12,NaN,Standard Scaler,NaN,...,2.686946,0.935130,c,Chemical + Physical,MLP_12,MLP,-1.768294,1,intermediate_model,69
15,Global Model,203,C,Chemical + Mineralogical + Physical,"(64225, 17)",NaN,MLP_1,NaN,Standard Scaler,NaN,...,2.747269,0.934495,c,Chemical + Physical,MLP_1,MLP,-1.748034,2,intermediate_model,66


<IPython.core.display.Javascript object>

In [119]:
df_sorted_topo_cols = df_sorted_topo[
    [
        "Model",
        # "Model_bkp",
        # "Model_bkp_2",
        "Timesteps",
        "RMSE_mean",
        "MAE_mean",
        "MAPE_mean",
        "R2_mean",
        "SCPM",
        "Dominated_Count",
        "Classification",
        "Dominates_Count",
        "Plant",
    ]
].round(2)

<IPython.core.display.Javascript object>

##### Top dominant models

In [120]:
# Top 4
df_sorted_topo_cols.sort_values(by="Dominates_Count", ascending=False).iloc[0:3]

,Model,Timesteps,RMSE_mean,MAE_mean,MAPE_mean,R2_mean,SCPM,Dominated_Count,Classification,Dominates_Count,Plant
20,MLP_6,NaN,1.51,1.12,2.63,0.94,-1.79,0,dominant_model,72,C
26,MLP_12,NaN,1.53,1.14,2.69,0.94,-1.77,1,intermediate_model,69,C
24,MLP_10,NaN,1.55,1.14,2.67,0.93,-1.76,1,intermediate_model,67,C


<IPython.core.display.Javascript object>

##### Top intermediate models

In [121]:
df_sorted_topo_cols[
    df_sorted_topo_cols["Classification"].eq("intermediate_model")
].sort_values(by="Dominates_Count", ascending=False).iloc[:2]

,Model,Timesteps,RMSE_mean,MAE_mean,MAPE_mean,R2_mean,SCPM,Dominated_Count,Classification,Dominates_Count,Plant
26,MLP_12,NaN,1.53,1.14,2.69,0.94,-1.77,1,intermediate_model,69,C
22,MLP_8,NaN,1.55,1.14,2.68,0.93,-1.76,1,intermediate_model,67,C


<IPython.core.display.Javascript object>

##### Top non dominant models

In [122]:
df_sorted_topo_cols[
    df_sorted_topo_cols["Classification"].eq("non_dominant_model")
].sort_values(by="Dominated_Count", ascending=True).iloc[:2]

,Model,Timesteps,RMSE_mean,MAE_mean,MAPE_mean,R2_mean,SCPM,Dominated_Count,Classification,Dominates_Count,Plant
5,LSTM6,14.0,24.25,23.44,53.32,-15.3,30.61,72,non_dominant_model,0,C


<IPython.core.display.Javascript object>

In [123]:
df_sorted_topo_models_cols = df_sorted_topo[
    [
        # "Model",
        # "Model_bkp",
        "Model_bkp_2",
        "Timesteps",
        "RMSE_mean",
        "MAE_mean",
        "MAPE_mean",
        "R2_mean",
        "SCPM",
        "Dominated_Count",
        "Classification",
        "Dominates_Count",
        "Plant",
    ]
]  # .round(2)

<IPython.core.display.Javascript object>

In [124]:
df_sorted_topo["Model_bkp_2"].value_counts()

Model_bkp_2
Conv1D         15
LSTM           15
Bi-LSTM        15
Transformer    15
MLP            13
Name: count, dtype: int64

<IPython.core.display.Javascript object>

In [125]:
df_sorted_topo_models_grouped = (
    df_sorted_topo_models_cols.groupby("Model_bkp_2").describe().T
)

<IPython.core.display.Javascript object>

In [126]:
# pd.set_option("display.max_rows", None)
# df_sorted_topo_models_grouped

<IPython.core.display.Javascript object>

In [127]:
df_sorted_topo_models_cols = df_sorted_topo[
    [
        "Model_bkp_2",
        "Dominated_Count",
        "Dominates_Count",
        "Classification",
    ]
]

<IPython.core.display.Javascript object>

In [128]:
df_sorted_topo_models_cols.groupby("Model_bkp_2").describe().round(2)

Dominated_Count                                              \
                      count   mean    std   min   25%   50%   75%   max   
Model_bkp_2                                                               
Bi-LSTM                15.0  48.80  14.27  25.0  35.5  54.0  60.5  68.0   
Conv1D                 15.0  17.80   5.52  11.0  14.0  16.0  20.0  30.0   
LSTM                   15.0  46.80  17.68  15.0  34.5  44.0  63.5  72.0   
MLP                    13.0   6.54   7.52   0.0   1.0   5.0   9.0  28.0   
Transformer            15.0  50.60  10.34  35.0  45.0  50.0  55.0  71.0   

            Dominates_Count                                              
                      count   mean    std   min   25%   50%   75%   max  
Model_bkp_2                                                              
Bi-LSTM                15.0  21.33  13.91   4.0  10.0  16.0  33.5  45.0  
Conv1D                 15.0  49.80   5.35  40.0  47.5  49.0  51.5  60.0  
LSTM                   15.0  23.47  17.07   0.0   7.0  28.0  35.0  53.0  
MLP                    13.0  63.54   7.85  40.0  63.0  66.0  67.0  72.0  
Transformer            15.0  20.00   9.82   1.0  15.5  21.0  25.5  35.0

<IPython.core.display.Javascript object>

In [129]:
df_sorted_topo_models_cols.groupby("Model_bkp_2")["Classification"].describe()

,count,unique,top,freq
Model_bkp_2,,,,
Bi-LSTM,15,1,intermediate_model,15
Conv1D,15,1,intermediate_model,15
LSTM,15,2,intermediate_model,14
MLP,13,2,intermediate_model,12
Transformer,15,1,intermediate_model,15


<IPython.core.display.Javascript object>

In [130]:
df_sorted_topo_models_cols.groupby("Model_bkp_2")[["Classification"]].value_counts()

Model_bkp_2  Classification    
Bi-LSTM      intermediate_model    15
Conv1D       intermediate_model    15
LSTM         intermediate_model    14
             non_dominant_model     1
MLP          intermediate_model    12
             dominant_model         1
Transformer  intermediate_model    15
Name: count, dtype: int64

<IPython.core.display.Javascript object>

In [131]:
(
    df_sorted_topo_models_cols.groupby("Model_bkp_2")[["Classification"]]
    .value_counts(normalize=True)
    .to_frame()
    * 100
).round(2)

proportion
Model_bkp_2 Classification                
Bi-LSTM     intermediate_model      100.00
Conv1D      intermediate_model      100.00
LSTM        intermediate_model       93.33
            non_dominant_model        6.67
MLP         intermediate_model       92.31
            dominant_model            7.69
Transformer intermediate_model      100.00

<IPython.core.display.Javascript object>

In [132]:
df_sorted_topo_models_cols

,Model_bkp_2,Dominated_Count,Dominates_Count,Classification
20,MLP,0,72,dominant_model
22,MLP,1,67,intermediate_model
24,MLP,1,67,intermediate_model
26,MLP,1,69,intermediate_model
15,MLP,2,66,intermediate_model
...,...,...,...,...
54,Bi-LSTM,68,4,intermediate_model
7,LSTM,69,2,intermediate_model
71,Transformer,69,2,intermediate_model
61,Transformer,71,1,intermediate_model


<IPython.core.display.Javascript object>

In [133]:
summary_stats = df_sorted_topo_models_cols.groupby("Model_bkp_2").agg(
    Total_Models=("Model_bkp_2", "count"),
    Mean_Dominance_Count=("Dominates_Count", "mean"),
    Median_Dominance_Count=("Dominates_Count", "median"),
    Total_Dominated=("Dominated_Count", "sum"),
    Total_Dominating=("Dominates_Count", "sum"),
)

total_models = summary_stats[
    "Total_Models"
].sum()  # Get the total number of models across all categories
summary_stats["Dominance Proportion"] = summary_stats["Total_Dominating"] / total_models

<IPython.core.display.Javascript object>

In [134]:
summary_stats

,Total_Models,Mean_Dominance_Count,Median_Dominance_Count,Total_Dominated,Total_Dominating,Dominance Proportion
Model_bkp_2,,,,,,
Bi-LSTM,15,21.333333,16.0,732,320,4.383562
Conv1D,15,49.800000,49.0,267,747,10.232877
LSTM,15,23.466667,28.0,702,352,4.821918
MLP,13,63.538462,66.0,85,826,11.315068
Transformer,15,20.000000,21.0,759,300,4.109589


<IPython.core.display.Javascript object>

In [135]:
df_sorted_topo_models_grouped.T

Timesteps                                                 \
                count      mean       std  min  25%  50%   75%   max   
Model_bkp_2                                                            
Bi-LSTM          15.0  7.333333  5.498918  1.0  1.0  7.0  14.0  14.0   
Conv1D           15.0  7.333333  5.498918  1.0  1.0  7.0  14.0  14.0   
LSTM             15.0  7.333333  5.498918  1.0  1.0  7.0  14.0  14.0   
MLP               0.0       NaN       NaN  NaN  NaN  NaN   NaN   NaN   
Transformer      15.0  7.333333  5.498918  1.0  1.0  7.0  14.0  14.0   

            RMSE_mean            ... Dominated_Count       Dominates_Count  \
                count      mean  ...             75%   max           count   
Model_bkp_2                      ...                                         
Bi-LSTM          15.0  3.427415  ...            60.5  68.0            15.0   
Conv1D           15.0  1.867428  ...            20.0  30.0            15.0   
LSTM             15.0  4.720149  ...            63.5  72.0            15.0   
MLP              13.0  1.613925  ...             9.0  28.0            13.0   
Transformer      15.0  3.498935  ...            55.0  71.0            15.0   

                                                                 
                  mean        std   min   25%   50%   75%   max  
Model_bkp_2                                                      
Bi-LSTM      21.333333  13.906148   4.0  10.0  16.0  33.5  45.0  
Conv1D       49.800000   5.347897  40.0  47.5  49.0  51.5  60.0  
LSTM         23.466667  17.070721   0.0   7.0  28.0  35.0  53.0  
MLP          63.538462   7.848730  40.0  63.0  66.0  67.0  72.0  
Transformer  20.000000   9.819805   1.0  15.5  21.0  25.5  35.0  

[5 rows x 64 columns]

<IPython.core.display.Javascript object>

### SCPM Analysis

In [136]:
cols = [
    "Model",
    # "Model_bkp",
    "Model_bkp_2",
    "Timesteps",
    "RMSE_mean",
    "MAE_mean",
    "MAPE_mean",
    "R2_mean",
    "SCPM",
    "Dominated_Count",
    "Dominates_Count",
    "Classification",
    "Plant",
]

<IPython.core.display.Javascript object>

In [137]:
df_sorted_scpm = df_sorted_topo.sort_values(by="SCPM").copy()

<IPython.core.display.Javascript object>

In [138]:
# (df_sorted_topo[cols].reset_index() == df_sorted_scpm[cols].reset_index()).all(axis=1)
df_sorted_scpm["topo_scmp_order_eq"] = df_sorted_topo.index == df_sorted_scpm.index

<IPython.core.display.Javascript object>

In [139]:
# df_sorted_topo[cols].round(2).head(5)

<IPython.core.display.Javascript object>

In [140]:
df_sorted_scpm[cols + ["topo_scmp_order_eq"]].round(2).head(7)

,Model,Model_bkp_2,Timesteps,RMSE_mean,MAE_mean,MAPE_mean,R2_mean,SCPM,Dominated_Count,Dominates_Count,Classification,Plant,topo_scmp_order_eq
20,MLP_6,MLP,NaN,1.51,1.12,2.63,0.94,-1.79,0,72,dominant_model,C,True
26,MLP_12,MLP,NaN,1.53,1.14,2.69,0.94,-1.77,1,69,intermediate_model,C,False
22,MLP_8,MLP,NaN,1.55,1.14,2.68,0.93,-1.76,1,67,intermediate_model,C,False
24,MLP_10,MLP,NaN,1.55,1.14,2.67,0.93,-1.76,1,67,intermediate_model,C,False
25,MLP_11,MLP,NaN,1.54,1.14,2.71,0.93,-1.76,2,67,intermediate_model,C,False
15,MLP_1,MLP,NaN,1.54,1.16,2.75,0.93,-1.75,2,66,intermediate_model,C,False
21,MLP_7,MLP,NaN,1.55,1.16,2.74,0.93,-1.75,5,66,intermediate_model,C,True


<IPython.core.display.Javascript object>

# Global Analysis (fine tuning results)

# Definitions

# Topological Analysis - NEW

## Definitions

In [141]:
# Function to compare two models strictly based on means only
def dominates(model1, model2):
    """Return True if model1 strictly dominates model2, False otherwise."""
    # Check if model1 is at least as good as model2 in all metrics
    at_least_as_good = (
        model1["RMSE_mean"] <= model2["RMSE_mean"]
        and model1["MAE_mean"] <= model2["MAE_mean"]
        and model1["MAPE_mean"] <= model2["MAPE_mean"]
        and model1["R2_mean"] >= model2["R2_mean"]
    )

    # Check if model1 is strictly better in at least one metric
    strictly_better_in_one = (
        model1["RMSE_mean"] < model2["RMSE_mean"]
        or model1["MAE_mean"] < model2["MAE_mean"]
        or model1["MAPE_mean"] < model2["MAPE_mean"]
        or model1["R2_mean"] > model2["R2_mean"]
    )

    # Model1 strictly dominates Model2 if it is at least as good in all metrics and
    # strictly better in at least one metric
    return at_least_as_good and strictly_better_in_one

<IPython.core.display.Javascript object>

In [142]:
def get_dominance_matrix_and_graph(df):
    n = len(df)
    dominance_matrix = np.zeros((n, n), dtype=bool)

    for i in range(n):
        for j in range(n):
            if i != j:
                dominance_matrix[i, j] = dominates(df.iloc[i], df.iloc[j])

    # Create the dominance graph
    dominance_graph = {i: [] for i in range(n)}
    for i in range(n):
        for j in range(n):
            if dominance_matrix[i, j]:
                dominance_graph[i].append(j)

    return dominance_matrix, dominance_graph

<IPython.core.display.Javascript object>

In [143]:
# Topological Sorting
def topological_sort(graph):
    """Perform topological sorting on the given graph."""
    in_degree = {u: 0 for u in graph}
    for u in graph:
        for v in graph[u]:
            in_degree[v] += 1

    queue = deque([u for u in graph if in_degree[u] == 0])
    topo_order = []

    while queue:
        u = queue.popleft()
        topo_order.append(u)

        for v in graph[u]:
            in_degree[v] -= 1
            if in_degree[v] == 0:
                queue.append(v)

    return topo_order

<IPython.core.display.Javascript object>

In [144]:
# Function to find the top models in each group
def find_top_models(group):
    max_net_dominance = group["Net_Dominance"].max()
    top_models = group[group["Net_Dominance"] == max_net_dominance]
    return top_models

<IPython.core.display.Javascript object>

### Dominance Analysis and SCPM

In [145]:
def compute_scpm(df):
    scaler = StandardScaler()
    standardized_metrics = scaler.fit_transform(
        df[["RMSE_mean", "MAE_mean", "MAPE_mean", "R2_mean"]]
    )

    # Creating a new dataframe with standardized values
    standardized_df = pd.DataFrame(
        standardized_metrics,
        columns=["RMSE_mean", "MAE_mean", "MAPE_mean", "R2_mean"],
    )

    # Summing all metrics and subtracting R2_mean
    standardized_df["Result"] = (
        standardized_df[["RMSE_mean", "MAE_mean", "MAPE_mean"]].sum(axis=1)
        - standardized_df["R2_mean"]
    )

    df["SCPM"] = standardized_df["Result"]
    return df

<IPython.core.display.Javascript object>

In [146]:
def make_dominance_analysis(df):
    # Compute dominance matrix and graph
    dominance_matrix, dominance_graph = get_dominance_matrix_and_graph(df)

    # Get the topological order
    topo_order = topological_sort(dominance_graph)
    df = df.copy()
    n = len(topo_order)

    # Identify Dominant Models
    dominant_models = [
        df["Model_bkp_2"].iloc[i]
        for i in range(len(df))
        if not np.any(dominance_matrix[:, i])
    ]

    # Identify Non-Dominant Models
    non_dominant_models = [
        df["Model_bkp_2"].iloc[i]
        for i in range(len(df))
        if np.any(dominance_matrix[:, i])
    ]

    # Identify Models that are both dominated and dominate at least one
    intermediate_models = [
        df["Model_bkp_2"].iloc[i]
        for i in range(len(df))
        if np.any(dominance_matrix[:, i]) and np.any(dominance_matrix[i, :])
    ]

    # Add count of times each model is dominated by others
    df["Dominated_Count"] = np.sum(dominance_matrix, axis=0)

    # Classify models
    df["Classification"] = "intermediate_model"  # default to intermediate_model
    df.loc[
        ~np.any(dominance_matrix, axis=1) & np.any(dominance_matrix, axis=0),
        "Classification",
    ] = "non_dominant_model"
    df.loc[~np.any(dominance_matrix, axis=0), "Classification"] = "dominant_model"

    # Map the topological order to the model names and get dominance counts
    sorted_models = [df["Model"].iloc[i] for i in topo_order]

    df_sorted = df.iloc[topo_order].copy()
    df_sorted["Dominates_Count"] = dominance_matrix.sum(axis=1)[topo_order]
    df_sorted_topo = df_sorted.copy()

    df_sorted = df_sorted.sort_values(by="Dominates_Count", ascending=False)
    df_sorted_count = df_sorted.copy()

    return {
        "dominance_matrix": dominance_matrix,
        "dominance_graph": dominance_graph,
        "df_sorted_topo": df_sorted_topo,
        "df_sorted_count": df_sorted_count,
    }

<IPython.core.display.Javascript object>

# Reading The files

In [147]:
csv_files_path_fine_tuning_full = dict()
csv_files_path_fine_tuning_grouped = dict()

<IPython.core.display.Javascript object>

## 203

### Plant C

In [148]:
path_fine_tuning_full = "/home/peressim/projects/ccs28-ml-modelling/reports/results/global_models/203/c/fine_tuning/full/"
path_fine_tuning_grouped = "/home/peressim/projects/ccs28-ml-modelling/reports/results/global_models/203/c/fine_tuning/grouped/"
plant = "c"

csv_files_path_fine_tuning_full = read_csv_files_path(
    csv_files_path_fine_tuning_full, path_fine_tuning_full, plant
)

csv_files_path_fine_tuning_grouped = read_csv_files_path(
    csv_files_path_fine_tuning_grouped, path_fine_tuning_grouped, plant
)

<IPython.core.display.Javascript object>

### Reading all data - pre training

#### Full

In [149]:
dfs_plant = dict()
column = ("plant", "")
pos = 2

for plant, csv_files in csv_files_path_fine_tuning_full.items():
    df = read_csv_files_full(csv_files, plant)
    df["plant"] = plant
    dfs_plant[plant] = df

<IPython.core.display.Javascript object>

In [150]:
df_fine_tuning_full = pd.concat(dfs_plant.values(), ignore_index=True)

<IPython.core.display.Javascript object>

In [151]:
df_fine_tuning_full.shape

(450, 23)

<IPython.core.display.Javascript object>

#### Grouped

In [152]:
dfs_plant = dict()
column = ("plant", "")
pos = 2

for plant, csv_files in csv_files_path_fine_tuning_grouped.items():
    df = read_csv_files_grouped(csv_files, plant)
    df["plant"] = plant
    df = preprocess_rename_columns(df)
    df = preprocess_change_columns_order(df, column, pos)
    dfs_plant[plant] = df

<IPython.core.display.Javascript object>

In [153]:
df_fine_tuning_grouped = pd.concat(dfs_plant.values(), ignore_index=True)

<IPython.core.display.Javascript object>

In [154]:
df_fine_tuning_grouped.shape

(30, 25)

<IPython.core.display.Javascript object>

In [155]:
df_copy = df_fine_tuning_grouped.copy()
df_copy = (
    df_copy.reset_index(level=0)
    .rename({"level_0": "Plant"}, axis=1)
    .reset_index(drop=True)
)

<IPython.core.display.Javascript object>

## Preprocessing steps

In [156]:
patterns = [
    "Chemical + Mineralogical + Feature Engineering",
    "Chemical + Mineralogical + CS7 + One-Hot",
    "Chemical + Mineralogical + Physical + One-Hot",
    "Chemical + Mineralogical + CS3 + One-Hot",
    "Chemical + Mineralogical + CS3 + CS7 + One-Hot",
    "Chemical + Mineralogical + One-Hot",
    "Chemical + Mineralogical + CS1 + CS3 + CS7 + One-Hot",
    "Chemical + Mineralogical + CS1 + CS3 + One-Hot",
    "Chemical + Mineralogical + CS1 + One-Hot",
    "Chemical + Mineralogical + CS2 + One-Hot",
    "Chemical + Feature Engineering",
]

replace_dict = {
    "Chemical + Mineralogical + CS3 + CS7": "Chemical + Mineralogical + Early CS",
    "Chemical + Mineralogical + CS7": "Chemical + Mineralogical + Early CS",
    "Chemical + Mineralogical + CS3": "Chemical + Mineralogical + Early CS",
    "Chemical + Mineralogical + CS1": "Chemical + Mineralogical + Early CS",
    "Chemical + Mineralogical + CS1 + CS3": "Chemical + Mineralogical + Early CS",
    "Chemical + Mineralogical + CS1 + CS3 + CS7": "Chemical + Mineralogical + Early CS",
    "Chemical + Mineralogical + CS2": "Chemical + Mineralogical + Early CS",
    "Chemical + CS1 + CS3 + CS7": "Chemical + Mineralogical + Early CS",
    "Chemical + CS3 + CS7": "Chemical + Mineralogical + Early CS",
    "Chemical + CS7": "Chemical + Mineralogical + Early CS",
    "Chemical + CS3": "Chemical + Mineralogical + Early CS",
    "Chemical + CS1 + CS3": "Chemical + Mineralogical + Early CS",
    "Chemical + CS1": "Chemical + Mineralogical + Early CS",
    "Chemical + CS1 + CS7": "Chemical + Mineralogical + Early CS",
}

replace_dict_2 = {
    "Chemical": "Chemical + Mineralogical",
    "Chemical + Properties CS Less": "Chemical + Mineralogical + Physical - Early CS",
    "Chemical + Mineralogical + Properties CS Less": "Chemical + Mineralogical + Physical - Early CS",
    "Chemical + Physical": "Chemical + Mineralogical + Physical",
}

df_copy = df_copy[~df_copy["Cross Validation"].eq("Out of time Split")].reset_index(
    drop=True
)

# Removing afterwards
df_copy = df_copy[~df_copy["Cross Validation"].eq("Repeated KFold")].reset_index(
    drop=True
)

df_copy = df_copy[~df_copy["Features"].apply(lambda x: x in patterns)].reset_index(
    drop=True
)
df_copy["Features_bkp"] = df_copy["Features"].copy()
df_copy["Features"] = df_copy["Features"].replace(replace_dict)

df_copy["Features"] = df_copy["Features"].replace(replace_dict_2)

# Removing afterwards
# df_copy = df_copy[
#     ~df_copy["Features"].eq("Chemical + Mineralogical + Early CS")
# ].reset_index(drop=True)

<IPython.core.display.Javascript object>

In [157]:
df_copy[("MAPE Train", "mean")] = df_copy[("MAPE Train", "mean")] * 100
df_copy[("MAPE Train", "std")] = df_copy[("MAPE Train", "std")] * 100
df_copy[("MAPE Test", "mean")] = df_copy[("MAPE Test", "mean")] * 100
df_copy[("MAPE Test", "std")] = df_copy[("MAPE Test", "std")] * 100

df_copy["Model_bkp"] = df_copy["Model"].copy()
df_copy["Model_bkp_2"] = df_copy["Model"] + df_copy["Timesteps"].apply(
    lambda x: "" if pd.isna(x) else "_" + str(x)
)
df_copy["Model"] = df_copy["Model"].replace(
    {
        "MLP": "Neural Networks",
        "LSTM": "Neural Networks",
        "GRU": "Neural Networks",
        "BidirectionalLSTM": "Neural Networks",
        "BidirectionalGRU": "Neural Networks",
        "Transformer": "Neural Networks",
        "Decision Tree": "Trees",
        "Random Forest": "Trees",
        "XGBoost": "Trees",
    }
)

<IPython.core.display.Javascript object>

In [158]:
df_copy["Cross Validation"].unique()

array(['Blocking Time Series Split', 'Time Series Split'], dtype=object)

<IPython.core.display.Javascript object>

In [159]:
df_copy["Features"].unique()

array(['Chemical + Mineralogical', 'Chemical + Mineralogical + Physical',
       'Chemical + Mineralogical + Physical - Early CS'], dtype=object)

<IPython.core.display.Javascript object>

In [160]:
df_copy["Model"].unique()

array(['Neural Networks', 'Conv1D'], dtype=object)

<IPython.core.display.Javascript object>

In [161]:
df_copy["Model_bkp"].unique()

array(['Transformer', 'MLP', 'BidirectionalLSTM', 'Conv1D', 'LSTM'],
      dtype=object)

<IPython.core.display.Javascript object>

In [162]:
df_copy["Model_bkp_2"].unique()

array(['Transformer_7.0', 'MLP', 'BidirectionalLSTM_7.0', 'Conv1D_7.0',
       'BidirectionalLSTM_1.0', 'LSTM_1.0', 'LSTM_7.0', 'Conv1D_14.0',
       'Transformer_1.0'], dtype=object)

<IPython.core.display.Javascript object>

In [163]:
df_copy.shape

(30, 29)

<IPython.core.display.Javascript object>

In [164]:
df_copy_grouped = df_copy.copy()

<IPython.core.display.Javascript object>

In [165]:
df_copy[
    [
        ("RMSE Test", "mean"),
        ("MAE Test", "mean"),
        ("MAPE Test", "mean"),
        ("R2 Test", "mean"),
    ]
].describe().round(2).T

,,count,mean,std,min,25%,50%,75%,max
RMSE Test,mean,30.0,3.22,2.27,1.08,1.70,2.52,3.49,10.23
MAE Test,mean,30.0,1.94,1.11,0.84,1.16,1.66,2.01,5.23
MAPE Test,mean,30.0,4.57,2.61,2.04,2.88,3.79,4.63,12.09
R2 Test,mean,30.0,0.38,1.12,-4.54,0.55,0.78,0.91,0.97


<IPython.core.display.Javascript object>

In [166]:
df_copy["plant"].unique()

array(['c'], dtype=object)

<IPython.core.display.Javascript object>

# Global Analysis (fine tuning results)

## ECICS - 203

In [167]:
df_ecics_grouped = df_copy.copy()

<IPython.core.display.Javascript object>

In [168]:
df_results = df_ecics_grouped[
    [
        ("Company", ""),
        ("plant", ""),
        ("Cross Validation", ""),
        ("Features", ""),
        ('Features_bkp', ''),
        ("Model", ""),
        ("Model_bkp", ""),         
        ('Model_bkp_2',''),
        ("Timesteps", ""),
        ("RMSE Test", "mean"),
        ("MAE Test", "mean"),
        ("MAPE Test", "mean"),
        ("R2 Test", "mean"),
        ("RMSE Test", "std"),
        ("MAE Test", "std"),
        ("MAPE Test", "std"),
        ("R2 Test", "std"),
    ]
].copy()
df_results = df_results.reset_index(drop=True)

# Rename columns
new_column_names = [
    "Company",
    "Plant",
    "Cross Validation",
    "Features",
    "Features_bkp",
    "Model",
    "Model_bkp",
    "Model_bkp_2",
    "Timesteps",
    "RMSE_mean",
    "MAE_mean",
    "MAPE_mean",
    "R2_mean",
    "RMSE_std",
    "MAE_std",
    "MAPE_std",
    "R2_std",
]

df_results.columns = new_column_names


<IPython.core.display.Javascript object>

In [169]:
df_results.shape

(30, 17)

<IPython.core.display.Javascript object>

In [170]:
df_results["Cross Validation"].unique()

array(['Blocking Time Series Split', 'Time Series Split'], dtype=object)

<IPython.core.display.Javascript object>

## Variable Grouping: CM

#### Analysis of the best models identified in the Local Analysis

**Best models for variable groupping CM were from:**

*BTiSS:*
    Dominance analysis: Plant C
    SCPM:Plant C

*TiSS:*
    Dominance analysis: Plant C
    SCPM:Plant C

In [171]:
df_results["Features"].unique()

array(['Chemical + Mineralogical', 'Chemical + Mineralogical + Physical',
       'Chemical + Mineralogical + Physical - Early CS'], dtype=object)

<IPython.core.display.Javascript object>

In [172]:
df_results_cm = (
    df_results[df_results["Features"].eq("Chemical + Mineralogical")]
    .copy()
    .reset_index(drop=True)
)

<IPython.core.display.Javascript object>

In [173]:
df_results_cm["Features"].unique()

array(['Chemical + Mineralogical'], dtype=object)

<IPython.core.display.Javascript object>

In [174]:
df_results_cm.shape

(10, 17)

<IPython.core.display.Javascript object>

### Plant C

In [175]:
df_results_cm = df_results_cm[df_results_cm["Plant"].eq("c")].reset_index(drop=True)

<IPython.core.display.Javascript object>

### Blocking time series

In [176]:
df_results_cm["Cross Validation"].unique()

array(['Blocking Time Series Split', 'Time Series Split'], dtype=object)

<IPython.core.display.Javascript object>

In [177]:
df_results_cm_btss = (
    df_results_cm[df_results_cm["Cross Validation"].eq("Blocking Time Series Split")]
    .copy()
    .reset_index(drop=True)
)

<IPython.core.display.Javascript object>

In [178]:
df_results_cm_btss["Cross Validation"].unique()

array(['Blocking Time Series Split'], dtype=object)

<IPython.core.display.Javascript object>

In [179]:
df_results_cm_btss.shape

(5, 17)

<IPython.core.display.Javascript object>

###### SCPM computation

In [180]:
df_results_cm_btss = compute_scpm(df_results_cm_btss)

<IPython.core.display.Javascript object>

##### Dominance Analysis

In [181]:
# Per project
dominance_dict = make_dominance_analysis(df_results_cm_btss)
dominance_matrix_cm_btss = dominance_dict["dominance_matrix"]
dominance_graph_cm_btss = dominance_dict["dominance_graph"]
df_sorted_count = dominance_dict["df_sorted_count"]
df_sorted_topo = dominance_dict["df_sorted_topo"]

<IPython.core.display.Javascript object>

In [182]:
dominance_matrix_cm_btss.shape, len(dominance_graph_cm_btss)

((5, 5), 5)

<IPython.core.display.Javascript object>

In [183]:
df_sorted_topo["Classification"].value_counts()

Classification
dominant_model        2
intermediate_model    2
non_dominant_model    1
Name: count, dtype: int64

<IPython.core.display.Javascript object>

In [184]:
df_sorted_topo.to_csv(
    "../../../../reports/results/global_models/ecics/ecics_203_c_dominance_analysis_cm_btss.csv",
    index=False,
)

<IPython.core.display.Javascript object>

In [185]:
df_sorted_topo.head(5)

,Company,Plant,Cross Validation,Features,Features_bkp,Model,Model_bkp,Model_bkp_2,Timesteps,RMSE_mean,...,MAPE_mean,R2_mean,RMSE_std,MAE_std,MAPE_std,R2_std,SCPM,Dominated_Count,Classification,Dominates_Count
0,203,c,Blocking Time Series Split,Chemical + Mineralogical,Chemical,Neural Networks,Transformer,Transformer_7.0,7.0,2.070881,...,3.262632,0.801263,1.701564,0.849215,2.633226,0.341221,-3.648681,0,dominant_model,3
3,203,c,Blocking Time Series Split,Chemical + Mineralogical,Chemical,Neural Networks,MLP,MLP,NaN,1.821639,...,3.070525,0.898877,0.535745,0.323934,0.746508,0.049300,-3.877936,0,dominant_model,3
2,203,c,Blocking Time Series Split,Chemical + Mineralogical,Chemical,Conv1D,Conv1D,Conv1D_7.0,7.0,2.967269,...,5.482018,0.712604,1.185527,0.891226,1.881392,0.217058,-2.002381,2,intermediate_model,2
1,203,c,Blocking Time Series Split,Chemical + Mineralogical,Chemical,Neural Networks,BidirectionalLSTM,BidirectionalLSTM_7.0,7.0,7.420992,...,11.029864,-0.881299,3.091809,1.712429,4.409763,1.507776,3.927759,3,intermediate_model,1
4,203,c,Blocking Time Series Split,Chemical + Mineralogical,Chemical,Neural Networks,LSTM,LSTM_7.0,7.0,8.108275,...,12.091657,-1.707786,5.529687,2.673904,6.124623,3.728722,5.601239,4,non_dominant_model,0


<IPython.core.display.Javascript object>

In [186]:
df_sorted_topo_cols = df_sorted_topo[
    [
        # "Model",
        # "Model_bkp",
        "Model_bkp_2",
        "RMSE_mean",
        "RMSE_std",
        "MAE_mean",
        "MAE_std",
        "MAPE_mean",
        "MAPE_std",
        "R2_mean",
        "R2_std",
        "SCPM",
        "Dominated_Count",
        "Classification",
        "Dominates_Count",
        "Plant",
    ]
].round(2)

<IPython.core.display.Javascript object>

###### Dominant Models

In [187]:
# Top 4
df_sorted_topo_cols.iloc[0:5]

,Model_bkp_2,RMSE_mean,RMSE_std,MAE_mean,MAE_std,MAPE_mean,MAPE_std,R2_mean,R2_std,SCPM,Dominated_Count,Classification,Dominates_Count,Plant
0,Transformer_7.0,2.07,1.70,1.28,0.85,3.26,2.63,0.80,0.34,-3.65,0,dominant_model,3,c
3,MLP,1.82,0.54,1.29,0.32,3.07,0.75,0.90,0.05,-3.88,0,dominant_model,3,c
2,Conv1D_7.0,2.97,1.19,2.37,0.89,5.48,1.88,0.71,0.22,-2.00,2,intermediate_model,2,c
1,BidirectionalLSTM_7.0,7.42,3.09,4.62,1.71,11.03,4.41,-0.88,1.51,3.93,3,intermediate_model,1,c
4,LSTM_7.0,8.11,5.53,5.23,2.67,12.09,6.12,-1.71,3.73,5.60,4,non_dominant_model,0,c


<IPython.core.display.Javascript object>

In [188]:
df_sorted_topo_cols[df_sorted_topo_cols["Classification"].eq("dominant_model")].shape

(2, 14)

<IPython.core.display.Javascript object>

###### Intermediate models

In [189]:
df_sorted_topo_cols[
    df_sorted_topo_cols["Classification"].eq("intermediate_model")
].sort_values(by=["Dominates_Count", "SCPM"], ascending=[False, True]).iloc[:3]

,Model_bkp_2,RMSE_mean,RMSE_std,MAE_mean,MAE_std,MAPE_mean,MAPE_std,R2_mean,R2_std,SCPM,Dominated_Count,Classification,Dominates_Count,Plant
2,Conv1D_7.0,2.97,1.19,2.37,0.89,5.48,1.88,0.71,0.22,-2.00,2,intermediate_model,2,c
1,BidirectionalLSTM_7.0,7.42,3.09,4.62,1.71,11.03,4.41,-0.88,1.51,3.93,3,intermediate_model,1,c


<IPython.core.display.Javascript object>

In [190]:
df_sorted_topo_cols[
    df_sorted_topo_cols["Classification"].eq("intermediate_model")
].shape

(2, 14)

<IPython.core.display.Javascript object>

###### non-dominant models

In [191]:
df_sorted_topo_cols[
    df_sorted_topo_cols["Classification"].eq("non_dominant_model")
].sort_values(by=["Dominates_Count", "SCPM"], ascending=[False, True]).iloc[:2]

,Model_bkp_2,RMSE_mean,RMSE_std,MAE_mean,MAE_std,MAPE_mean,MAPE_std,R2_mean,R2_std,SCPM,Dominated_Count,Classification,Dominates_Count,Plant
4,LSTM_7.0,8.11,5.53,5.23,2.67,12.09,6.12,-1.71,3.73,5.6,4,non_dominant_model,0,c


<IPython.core.display.Javascript object>

In [192]:
df_sorted_topo_cols[
    df_sorted_topo_cols["Classification"].eq("non_dominant_model")
].shape

(1, 14)

<IPython.core.display.Javascript object>

###### Statistics per model type

In [193]:
df_sorted_topo_models_cols = df_sorted_topo[
    [
        "Model",
        # "Model_bkp",
        # "Model_bkp_2",
        "RMSE_mean",
        "RMSE_std",
        "MAE_mean",
        "MAE_std",
        "MAPE_mean",
        "MAPE_std",
        "R2_mean",
        "R2_std",
        "SCPM",
        "Dominated_Count",
        "Dominates_Count",
        "Classification",
        "Plant",
    ]
]  # .round(2)

<IPython.core.display.Javascript object>

In [194]:
df_sorted_topo["Model"].value_counts()

Model
Neural Networks    4
Conv1D             1
Name: count, dtype: int64

<IPython.core.display.Javascript object>

In [195]:
df_sorted_topo_models_grouped = df_sorted_topo_models_cols.groupby("Model").describe().T

<IPython.core.display.Javascript object>

In [196]:
# pd.set_option("display.max_rows", None)
# df_sorted_topo_models_grouped

<IPython.core.display.Javascript object>

In [197]:
df_sorted_topo_models_cols = df_sorted_topo[
    [
        "Model",
        "Dominated_Count",
        "Dominates_Count",
        "Classification",
    ]
]

<IPython.core.display.Javascript object>

In [198]:
(
    df_sorted_topo_models_cols.groupby("Model")[["Classification"]]
    .value_counts(normalize=True)
    .to_frame()
    * 100
).round(2)

proportion
Model           Classification                
Conv1D          intermediate_model       100.0
Neural Networks dominant_model            50.0
                intermediate_model        25.0
                non_dominant_model        25.0

<IPython.core.display.Javascript object>

In [199]:
df_sorted_topo_models_cols.groupby("Model").describe().round(2).T

Model                  Conv1D  Neural Networks
Dominated_Count count     1.0             4.00
                mean      2.0             1.75
                std       NaN             2.06
                min       2.0             0.00
                25%       2.0             0.00
                50%       2.0             1.50
                75%       2.0             3.25
                max       2.0             4.00
Dominates_Count count     1.0             4.00
                mean      2.0             1.75
                std       NaN             1.50
                min       2.0             0.00
                25%       2.0             0.75
                50%       2.0             2.00
                75%       2.0             3.00
                max       2.0             3.00

<IPython.core.display.Javascript object>

In [200]:
# df_sorted_topo_models_cols

<IPython.core.display.Javascript object>

In [201]:
df_sorted_topo_models_cols.groupby("Model")["Classification"].describe()

,count,unique,top,freq
Model,,,,
Conv1D,1,1,intermediate_model,1
Neural Networks,4,3,dominant_model,2


<IPython.core.display.Javascript object>

In [202]:
df_sorted_topo_models_cols.groupby("Model")[["Classification"]].value_counts()

Model            Classification    
Conv1D           intermediate_model    1
Neural Networks  dominant_model        2
                 intermediate_model    1
                 non_dominant_model    1
Name: count, dtype: int64

<IPython.core.display.Javascript object>

In [203]:
# df_sorted_topo_models_cols

<IPython.core.display.Javascript object>

In [204]:
summary_stats = df_sorted_topo_models_cols.groupby("Model").agg(
    Total_Models=("Model", "count"),
    Mean_Dominance_Count=("Dominates_Count", "mean"),
    Median_Dominance_Count=("Dominates_Count", "median"),
    Total_Dominated=("Dominated_Count", "sum"),
    Total_Dominating=("Dominates_Count", "sum"),
)

total_models = summary_stats[
    "Total_Models"
].sum()  # Get the total number of models across all categories
summary_stats["Dominance Proportion"] = summary_stats["Total_Dominating"] / total_models

<IPython.core.display.Javascript object>

In [205]:
summary_stats

,Total_Models,Mean_Dominance_Count,Median_Dominance_Count,Total_Dominated,Total_Dominating,Dominance Proportion
Model,,,,,,
Conv1D,1,2.00,2.0,2,2,0.4
Neural Networks,4,1.75,2.0,7,7,1.4


<IPython.core.display.Javascript object>

In [206]:
df_sorted_topo_models_grouped.T

RMSE_mean                                                    \
                    count      mean       std       min       25%       50%   
Model                                                                         
Conv1D                1.0  2.967269       NaN  2.967269  2.967269  2.967269   
Neural Networks       4.0  4.855447  3.372472  1.821639  2.008570  4.745936   

                                    RMSE_std            ... Dominated_Count  \
                      75%       max    count      mean  ...             75%   
Model                                                   ...                   
Conv1D           2.967269  2.967269      1.0  1.185527  ...            2.00   
Neural Networks  7.592813  8.108275      4.0  2.714701  ...            3.25   

                     Dominates_Count                                       
                 max           count  mean  std  min   25%  50%  75%  max  
Model                                                                      
Conv1D           2.0             1.0  2.00  NaN  2.0  2.00  2.0  2.0  2.0  
Neural Networks  4.0             4.0  1.75  1.5  0.0  0.75  2.0  3.0  3.0  

[2 rows x 88 columns]

<IPython.core.display.Javascript object>

### SCPM Analysis

In [207]:
cols = [
    # "Model",
    # "Model_bkp",
    "Model_bkp_2",
    "RMSE_mean",
    "RMSE_std",
    "MAE_mean",
    "MAE_std",
    "MAPE_mean",
    "MAPE_std",
    "R2_mean",
    "R2_std",
    "SCPM",
    "Dominated_Count",
    "Dominates_Count",
    "Classification",
    "Plant",
]

<IPython.core.display.Javascript object>

#### SCPM Analysis by Project

In [208]:
df_sorted_scpm = df_sorted_topo.sort_values(by="SCPM").copy()

<IPython.core.display.Javascript object>

In [209]:
# (df_sorted_topo[cols].reset_index() == df_sorted_scpm[cols].reset_index()).all(axis=1)
df_sorted_scpm["topo_scmp_order_eq"] = df_sorted_topo.index == df_sorted_scpm.index

<IPython.core.display.Javascript object>

In [210]:
df_sorted_scpm[cols + ["topo_scmp_order_eq"]].round(2).head(5)

,Model_bkp_2,RMSE_mean,RMSE_std,MAE_mean,MAE_std,MAPE_mean,MAPE_std,R2_mean,R2_std,SCPM,Dominated_Count,Dominates_Count,Classification,Plant,topo_scmp_order_eq
3,MLP,1.82,0.54,1.29,0.32,3.07,0.75,0.90,0.05,-3.88,0,3,dominant_model,c,False
0,Transformer_7.0,2.07,1.70,1.28,0.85,3.26,2.63,0.80,0.34,-3.65,0,3,dominant_model,c,False
2,Conv1D_7.0,2.97,1.19,2.37,0.89,5.48,1.88,0.71,0.22,-2.00,2,2,intermediate_model,c,True
1,BidirectionalLSTM_7.0,7.42,3.09,4.62,1.71,11.03,4.41,-0.88,1.51,3.93,3,1,intermediate_model,c,True
4,LSTM_7.0,8.11,5.53,5.23,2.67,12.09,6.12,-1.71,3.73,5.60,4,0,non_dominant_model,c,True


<IPython.core.display.Javascript object>

### Time Series Split

In [211]:
df_results_cm["Cross Validation"].unique()

array(['Blocking Time Series Split', 'Time Series Split'], dtype=object)

<IPython.core.display.Javascript object>

In [212]:
df_results_cm_tss = (
    df_results_cm[df_results_cm["Cross Validation"].eq("Time Series Split")]
    .copy()
    .reset_index(drop=True)
)

<IPython.core.display.Javascript object>

In [213]:
df_results_cm_tss["Cross Validation"].unique()

array(['Time Series Split'], dtype=object)

<IPython.core.display.Javascript object>

In [214]:
df_results_cm_tss.shape

(5, 17)

<IPython.core.display.Javascript object>

###### SCPM computation

In [215]:
df_results_cm_tss = compute_scpm(df_results_cm_tss)

<IPython.core.display.Javascript object>

##### Dominance Analysis

In [216]:
# Per project
dominance_dict = make_dominance_analysis(df_results_cm_tss)
dominance_matrix_cm_tss = dominance_dict["dominance_matrix"]
dominance_graph_cm_tss = dominance_dict["dominance_graph"]
df_sorted_count = dominance_dict["df_sorted_count"]
df_sorted_topo = dominance_dict["df_sorted_topo"]

<IPython.core.display.Javascript object>

In [217]:
dominance_matrix_cm_tss.shape, len(dominance_graph_cm_tss)

((5, 5), 5)

<IPython.core.display.Javascript object>

In [218]:
df_sorted_topo["Classification"].value_counts()

Classification
dominant_model        2
intermediate_model    2
non_dominant_model    1
Name: count, dtype: int64

<IPython.core.display.Javascript object>

In [219]:
df_sorted_topo.to_csv(
    "../../../../reports/results/global_models/ecics/ecics_203_c_dominance_analysis_cm_tss.csv",
    index=False,
)

<IPython.core.display.Javascript object>

In [220]:
df_sorted_topo.head(5)

,Company,Plant,Cross Validation,Features,Features_bkp,Model,Model_bkp,Model_bkp_2,Timesteps,RMSE_mean,...,MAPE_mean,R2_mean,RMSE_std,MAE_std,MAPE_std,R2_std,SCPM,Dominated_Count,Classification,Dominates_Count
0,203,c,Time Series Split,Chemical + Mineralogical,Chemical,Neural Networks,Transformer,Transformer_7.0,7.0,3.001804,...,4.473344,0.729180,0.653006,0.386746,1.003257,0.112467,-4.278989,0,dominant_model,3
3,203,c,Time Series Split,Chemical + Mineralogical,Chemical,Neural Networks,MLP,MLP,NaN,2.903118,...,4.537533,0.734415,0.951223,0.742230,1.485028,0.182382,-4.192899,0,dominant_model,3
2,203,c,Time Series Split,Chemical + Mineralogical,Chemical,Conv1D,Conv1D,Conv1D_7.0,7.0,3.967133,...,6.546697,0.504643,1.155920,0.936504,2.038555,0.253425,-0.719298,2,intermediate_model,2
1,203,c,Time Series Split,Chemical + Mineralogical,Chemical,Neural Networks,BidirectionalLSTM,BidirectionalLSTM_7.0,7.0,5.749278,...,8.555478,-0.100069,1.728445,0.922098,2.248478,0.825061,4.061361,3,intermediate_model,1
4,203,c,Time Series Split,Chemical + Mineralogical,Chemical,Neural Networks,LSTM,LSTM_7.0,7.0,6.130604,...,8.927445,-0.248383,2.117707,1.123868,2.502762,0.830469,5.129825,4,non_dominant_model,0


<IPython.core.display.Javascript object>

In [221]:
df_sorted_topo_cols = df_sorted_topo[
    [
        # "Model",
        # "Model_bkp",
        "Model_bkp_2",
        "RMSE_mean",
        "RMSE_std",
        "MAE_mean",
        "MAE_std",
        "MAPE_mean",
        "MAPE_std",
        "R2_mean",
        "R2_std",
        "SCPM",
        "Dominated_Count",
        "Classification",
        "Dominates_Count",
        "Plant",
    ]
]  # .round(2)

<IPython.core.display.Javascript object>

###### Dominant Models

In [222]:
# Top 4
df_sorted_topo_cols.iloc[0:2].sort_values(by="Dominates_Count", ascending=False).round(
    2
)

,Model_bkp_2,RMSE_mean,RMSE_std,MAE_mean,MAE_std,MAPE_mean,MAPE_std,R2_mean,R2_std,SCPM,Dominated_Count,Classification,Dominates_Count,Plant
0,Transformer_7.0,3.0,0.65,1.88,0.39,4.47,1.00,0.73,0.11,-4.28,0,dominant_model,3,c
3,MLP,2.9,0.95,1.99,0.74,4.54,1.49,0.73,0.18,-4.19,0,dominant_model,3,c


<IPython.core.display.Javascript object>

In [223]:
df_sorted_topo_cols[df_sorted_topo_cols["Classification"].eq("dominant_model")].shape

(2, 14)

<IPython.core.display.Javascript object>

###### Intermediate models

In [224]:
df_sorted_topo_cols[
    df_sorted_topo_cols["Classification"].eq("intermediate_model")
].sort_values(by=["Dominates_Count", "SCPM"], ascending=[False, True]).iloc[:4].round(2)

,Model_bkp_2,RMSE_mean,RMSE_std,MAE_mean,MAE_std,MAPE_mean,MAPE_std,R2_mean,R2_std,SCPM,Dominated_Count,Classification,Dominates_Count,Plant
2,Conv1D_7.0,3.97,1.16,2.84,0.94,6.55,2.04,0.5,0.25,-0.72,2,intermediate_model,2,c
1,BidirectionalLSTM_7.0,5.75,1.73,3.59,0.92,8.56,2.25,-0.1,0.83,4.06,3,intermediate_model,1,c


<IPython.core.display.Javascript object>

In [225]:
df_sorted_topo_cols[
    df_sorted_topo_cols["Classification"].eq("intermediate_model")
].shape

(2, 14)

<IPython.core.display.Javascript object>

###### non-dominant models

In [226]:
df_sorted_topo_cols[
    df_sorted_topo_cols["Classification"].eq("non_dominant_model")
].sort_values(by=["Dominates_Count", "SCPM"], ascending=[False, True]).iloc[:3]

,Model_bkp_2,RMSE_mean,RMSE_std,MAE_mean,MAE_std,MAPE_mean,MAPE_std,R2_mean,R2_std,SCPM,Dominated_Count,Classification,Dominates_Count,Plant
4,LSTM_7.0,6.130604,2.117707,3.780291,1.123868,8.927445,2.502762,-0.248383,0.830469,5.129825,4,non_dominant_model,0,c


<IPython.core.display.Javascript object>

In [227]:
df_sorted_topo_cols[
    df_sorted_topo_cols["Classification"].eq("non_dominant_model")
].shape

(1, 14)

<IPython.core.display.Javascript object>

###### Statistics per model type

In [228]:
df_sorted_topo_models_cols = df_sorted_topo[
    [
        "Model",
        # "Model_bkp",
        # "Model_bkp_2",
        "RMSE_mean",
        "RMSE_std",
        "MAE_mean",
        "MAE_std",
        "MAPE_mean",
        "MAPE_std",
        "R2_mean",
        "R2_std",
        "SCPM",
        "Dominated_Count",
        "Dominates_Count",
        "Classification",
        "Plant",
    ]
]  # .round(2)

<IPython.core.display.Javascript object>

In [229]:
df_sorted_topo["Model"].value_counts()

Model
Neural Networks    4
Conv1D             1
Name: count, dtype: int64

<IPython.core.display.Javascript object>

In [230]:
df_sorted_topo_models_grouped = df_sorted_topo_models_cols.groupby("Model").describe().T

<IPython.core.display.Javascript object>

In [231]:
df_sorted_topo_models_cols = df_sorted_topo[
    [
        "Model",
        "Dominated_Count",
        "Dominates_Count",
        "Classification",
    ]
]

<IPython.core.display.Javascript object>

In [232]:
(
    df_sorted_topo_models_cols.groupby("Model")[["Classification"]]
    .value_counts(normalize=True)
    .to_frame()
    * 100
).round(2)

proportion
Model           Classification                
Conv1D          intermediate_model       100.0
Neural Networks dominant_model            50.0
                intermediate_model        25.0
                non_dominant_model        25.0

<IPython.core.display.Javascript object>

In [233]:
df_sorted_topo_models_cols.groupby("Model").describe().round(2).T

Model                  Conv1D  Neural Networks
Dominated_Count count     1.0             4.00
                mean      2.0             1.75
                std       NaN             2.06
                min       2.0             0.00
                25%       2.0             0.00
                50%       2.0             1.50
                75%       2.0             3.25
                max       2.0             4.00
Dominates_Count count     1.0             4.00
                mean      2.0             1.75
                std       NaN             1.50
                min       2.0             0.00
                25%       2.0             0.75
                50%       2.0             2.00
                75%       2.0             3.00
                max       2.0             3.00

<IPython.core.display.Javascript object>

In [234]:
# df_sorted_topo_models_cols

<IPython.core.display.Javascript object>

In [235]:
df_sorted_topo_models_cols.groupby("Model")["Classification"].describe()

,count,unique,top,freq
Model,,,,
Conv1D,1,1,intermediate_model,1
Neural Networks,4,3,dominant_model,2


<IPython.core.display.Javascript object>

In [236]:
df_sorted_topo_models_cols.groupby("Model")[["Classification"]].value_counts()

Model            Classification    
Conv1D           intermediate_model    1
Neural Networks  dominant_model        2
                 intermediate_model    1
                 non_dominant_model    1
Name: count, dtype: int64

<IPython.core.display.Javascript object>

In [237]:
summary_stats = df_sorted_topo_models_cols.groupby("Model").agg(
    Total_Models=("Model", "count"),
    Mean_Dominance_Count=("Dominates_Count", "mean"),
    Median_Dominance_Count=("Dominates_Count", "median"),
    Total_Dominated=("Dominated_Count", "sum"),
    Total_Dominating=("Dominates_Count", "sum"),
)

total_models = summary_stats[
    "Total_Models"
].sum()  # Get the total number of models across all categories
summary_stats["Dominance Proportion"] = summary_stats["Total_Dominating"] / total_models

<IPython.core.display.Javascript object>

In [238]:
summary_stats.round(2)

,Total_Models,Mean_Dominance_Count,Median_Dominance_Count,Total_Dominated,Total_Dominating,Dominance Proportion
Model,,,,,,
Conv1D,1,2.00,2.0,2,2,0.4
Neural Networks,4,1.75,2.0,7,7,1.4


<IPython.core.display.Javascript object>

In [239]:
df_sorted_topo_models_grouped.T

RMSE_mean                                                    \
                    count      mean       std       min       25%       50%   
Model                                                                         
Conv1D                1.0  3.967133       NaN  3.967133  3.967133  3.967133   
Neural Networks       4.0  4.446201  1.732302  2.903118  2.977132  4.375541   

                                    RMSE_std            ... Dominated_Count  \
                      75%       max    count      mean  ...             75%   
Model                                                   ...                   
Conv1D           3.967133  3.967133      1.0  1.155920  ...            2.00   
Neural Networks  5.844610  6.130604      4.0  1.362595  ...            3.25   

                     Dominates_Count                                       
                 max           count  mean  std  min   25%  50%  75%  max  
Model                                                                      
Conv1D           2.0             1.0  2.00  NaN  2.0  2.00  2.0  2.0  2.0  
Neural Networks  4.0             4.0  1.75  1.5  0.0  0.75  2.0  3.0  3.0  

[2 rows x 88 columns]

<IPython.core.display.Javascript object>

### SCPM Analysis

In [240]:
cols = [
    # "Model",
    # "Model_bkp",
    "Model_bkp_2",
    "RMSE_mean",
    "RMSE_std",
    "MAE_mean",
    "MAE_std",
    "MAPE_mean",
    "MAPE_std",
    "R2_mean",
    "R2_std",
    "SCPM",
    "Dominated_Count",
    "Dominates_Count",
    "Classification",
    "Plant",
]

<IPython.core.display.Javascript object>

#### SCPM Analysis by Project

In [241]:
df_sorted_scpm = df_sorted_topo.sort_values(by="SCPM").copy()

<IPython.core.display.Javascript object>

In [242]:
# (df_sorted_topo[cols].reset_index() == df_sorted_scpm[cols].reset_index()).all(axis=1)
df_sorted_scpm["topo_scmp_order_eq"] = df_sorted_topo.index == df_sorted_scpm.index

<IPython.core.display.Javascript object>

In [243]:
df_sorted_scpm[cols + ["topo_scmp_order_eq"]].round(2).head(5)

,Model_bkp_2,RMSE_mean,RMSE_std,MAE_mean,MAE_std,MAPE_mean,MAPE_std,R2_mean,R2_std,SCPM,Dominated_Count,Dominates_Count,Classification,Plant,topo_scmp_order_eq
0,Transformer_7.0,3.00,0.65,1.88,0.39,4.47,1.00,0.73,0.11,-4.28,0,3,dominant_model,c,True
3,MLP,2.90,0.95,1.99,0.74,4.54,1.49,0.73,0.18,-4.19,0,3,dominant_model,c,True
2,Conv1D_7.0,3.97,1.16,2.84,0.94,6.55,2.04,0.50,0.25,-0.72,2,2,intermediate_model,c,True
1,BidirectionalLSTM_7.0,5.75,1.73,3.59,0.92,8.56,2.25,-0.10,0.83,4.06,3,1,intermediate_model,c,True
4,LSTM_7.0,6.13,2.12,3.78,1.12,8.93,2.50,-0.25,0.83,5.13,4,0,non_dominant_model,c,True


<IPython.core.display.Javascript object>

## Variable Grouping: CM-P

#### Analysis of the best models identified in the Local Analysis

**Best models for variable groupping CM-P were from:**

*BTiSS:*
    Dominance analysis: Plant C
    SCPM:Plant C

*TiSS:*
    Dominance analysis: Plant C
    SCPM:Plant C

In [244]:
df_results["Features"].unique()

array(['Chemical + Mineralogical', 'Chemical + Mineralogical + Physical',
       'Chemical + Mineralogical + Physical - Early CS'], dtype=object)

<IPython.core.display.Javascript object>

In [245]:
df_results_cm_p = (
    df_results[
        df_results["Features"].eq("Chemical + Mineralogical + Physical - Early CS")
    ]
    .copy()
    .reset_index(drop=True)
)

<IPython.core.display.Javascript object>

In [246]:
df_results_cm_p["Features"].unique()

array(['Chemical + Mineralogical + Physical - Early CS'], dtype=object)

<IPython.core.display.Javascript object>

In [247]:
df_results_cm_p.shape

(10, 17)

<IPython.core.display.Javascript object>

### Plant C

In [248]:
df_results_cm_p = df_results_cm_p[df_results_cm_p["Plant"].eq("c")].reset_index(
    drop=True
)

<IPython.core.display.Javascript object>

### Blocking time series

In [249]:
df_results_cm_p["Cross Validation"].unique()

array(['Blocking Time Series Split', 'Time Series Split'], dtype=object)

<IPython.core.display.Javascript object>

In [250]:
df_results_cm_p_btss = (
    df_results_cm_p[
        df_results_cm_p["Cross Validation"].eq("Blocking Time Series Split")
    ]
    .copy()
    .reset_index(drop=True)
)

<IPython.core.display.Javascript object>

In [251]:
df_results_cm_p_btss["Cross Validation"].unique()

array(['Blocking Time Series Split'], dtype=object)

<IPython.core.display.Javascript object>

In [252]:
df_results_cm_p_btss.shape

(5, 17)

<IPython.core.display.Javascript object>

###### SCPM computation

In [253]:
df_results_cm_p_btss = compute_scpm(df_results_cm_p_btss)

<IPython.core.display.Javascript object>

##### Dominance Analysis

In [254]:
# Per project
dominance_dict = make_dominance_analysis(df_results_cm_p_btss)
dominance_matrix_cm_p_btss = dominance_dict["dominance_matrix"]
dominance_graph_cm_p_btss = dominance_dict["dominance_graph"]
df_sorted_count = dominance_dict["df_sorted_count"]
df_sorted_topo = dominance_dict["df_sorted_topo"]

<IPython.core.display.Javascript object>

In [255]:
dominance_matrix_cm_p_btss.shape, len(dominance_graph_cm_p_btss)

((5, 5), 5)

<IPython.core.display.Javascript object>

In [256]:
df_sorted_topo["Classification"].value_counts()

Classification
intermediate_model    3
dominant_model        1
non_dominant_model    1
Name: count, dtype: int64

<IPython.core.display.Javascript object>

In [257]:
df_sorted_topo.to_csv(
    "../../../../reports/results/global_models/ecics/ecics_203_c_dominance_analysis_cm_p_btss.csv",
    index=False,
)

<IPython.core.display.Javascript object>

In [258]:
df_sorted_topo.head(5)

,Company,Plant,Cross Validation,Features,Features_bkp,Model,Model_bkp,Model_bkp_2,Timesteps,RMSE_mean,...,MAPE_mean,R2_mean,RMSE_std,MAE_std,MAPE_std,R2_std,SCPM,Dominated_Count,Classification,Dominates_Count
4,203,c,Blocking Time Series Split,Chemical + Mineralogical + Physical - Early CS,Chemical + Properties CS Less,Neural Networks,Transformer,Transformer_1.0,1.0,1.375060,...,2.090569,0.943194,0.296677,0.146197,0.293774,0.021891,-3.267481,0,dominant_model,4
0,203,c,Blocking Time Series Split,Chemical + Mineralogical + Physical - Early CS,Chemical + Properties CS Less,Neural Networks,MLP,MLP,NaN,1.509755,...,2.623906,0.933690,0.248915,0.133875,0.273091,0.017394,-1.466098,1,intermediate_model,1
2,203,c,Blocking Time Series Split,Chemical + Mineralogical + Physical - Early CS,Chemical + Properties CS Less,Neural Networks,LSTM,LSTM_1.0,1.0,1.499910,...,2.547190,0.933078,0.297040,0.098587,0.237979,0.026938,-1.595288,1,intermediate_model,1
3,203,c,Blocking Time Series Split,Chemical + Mineralogical + Physical - Early CS,Chemical + Properties CS Less,Conv1D,Conv1D,Conv1D_7.0,7.0,1.420810,...,2.674327,0.940179,0.111836,0.069613,0.169577,0.006047,-1.489264,1,intermediate_model,1
1,203,c,Blocking Time Series Split,Chemical + Mineralogical + Physical - Early CS,Chemical + Properties CS Less,Neural Networks,BidirectionalLSTM,BidirectionalLSTM_1.0,1.0,2.639592,...,4.177795,0.714920,1.806680,1.102736,2.711727,0.404196,7.818132,4,non_dominant_model,0


<IPython.core.display.Javascript object>

In [259]:
df_sorted_topo_cols = df_sorted_topo[
    [
        # "Model",
        # "Model_bkp",
        "Model_bkp_2",
        "RMSE_mean",
        "RMSE_std",
        "MAE_mean",
        "MAE_std",
        "MAPE_mean",
        "MAPE_std",
        "R2_mean",
        "R2_std",
        "SCPM",
        "Dominated_Count",
        "Classification",
        "Dominates_Count",
        "Plant",
    ]
].round(2)

<IPython.core.display.Javascript object>

###### Dominant Models

In [260]:
# Top 4
df_sorted_topo_cols.iloc[0:5]

,Model_bkp_2,RMSE_mean,RMSE_std,MAE_mean,MAE_std,MAPE_mean,MAPE_std,R2_mean,R2_std,SCPM,Dominated_Count,Classification,Dominates_Count,Plant
4,Transformer_1.0,1.38,0.30,0.87,0.15,2.09,0.29,0.94,0.02,-3.27,0,dominant_model,4,c
0,MLP,1.51,0.25,1.07,0.13,2.62,0.27,0.93,0.02,-1.47,1,intermediate_model,1,c
2,LSTM_1.0,1.50,0.30,1.06,0.10,2.55,0.24,0.93,0.03,-1.60,1,intermediate_model,1,c
3,Conv1D_7.0,1.42,0.11,1.12,0.07,2.67,0.17,0.94,0.01,-1.49,1,intermediate_model,1,c
1,BidirectionalLSTM_1.0,2.64,1.81,1.74,1.10,4.18,2.71,0.71,0.40,7.82,4,non_dominant_model,0,c


<IPython.core.display.Javascript object>

In [261]:
df_sorted_topo_cols[df_sorted_topo_cols["Classification"].eq("dominant_model")].shape

(1, 14)

<IPython.core.display.Javascript object>

###### Intermediate models

In [262]:
df_sorted_topo_cols[
    df_sorted_topo_cols["Classification"].eq("intermediate_model")
].sort_values(by=["Dominates_Count", "SCPM"], ascending=[False, True]).iloc[:3]

,Model_bkp_2,RMSE_mean,RMSE_std,MAE_mean,MAE_std,MAPE_mean,MAPE_std,R2_mean,R2_std,SCPM,Dominated_Count,Classification,Dominates_Count,Plant
2,LSTM_1.0,1.50,0.30,1.06,0.10,2.55,0.24,0.93,0.03,-1.60,1,intermediate_model,1,c
3,Conv1D_7.0,1.42,0.11,1.12,0.07,2.67,0.17,0.94,0.01,-1.49,1,intermediate_model,1,c
0,MLP,1.51,0.25,1.07,0.13,2.62,0.27,0.93,0.02,-1.47,1,intermediate_model,1,c


<IPython.core.display.Javascript object>

In [263]:
df_sorted_topo_cols[
    df_sorted_topo_cols["Classification"].eq("intermediate_model")
].shape

(3, 14)

<IPython.core.display.Javascript object>

###### non-dominant models

In [264]:
df_sorted_topo_cols[
    df_sorted_topo_cols["Classification"].eq("non_dominant_model")
].sort_values(by=["Dominates_Count", "SCPM"], ascending=[False, True]).iloc[:2]

,Model_bkp_2,RMSE_mean,RMSE_std,MAE_mean,MAE_std,MAPE_mean,MAPE_std,R2_mean,R2_std,SCPM,Dominated_Count,Classification,Dominates_Count,Plant
1,BidirectionalLSTM_1.0,2.64,1.81,1.74,1.1,4.18,2.71,0.71,0.4,7.82,4,non_dominant_model,0,c


<IPython.core.display.Javascript object>

In [265]:
df_sorted_topo_cols[
    df_sorted_topo_cols["Classification"].eq("non_dominant_model")
].shape

(1, 14)

<IPython.core.display.Javascript object>

###### Statistics per model type

In [266]:
df_sorted_topo_models_cols = df_sorted_topo[
    [
        "Model",
        # "Model_bkp",
        # "Model_bkp_2",
        "RMSE_mean",
        "RMSE_std",
        "MAE_mean",
        "MAE_std",
        "MAPE_mean",
        "MAPE_std",
        "R2_mean",
        "R2_std",
        "SCPM",
        "Dominated_Count",
        "Dominates_Count",
        "Classification",
        "Plant",
    ]
]  # .round(2)

<IPython.core.display.Javascript object>

In [267]:
df_sorted_topo["Model"].value_counts()

Model
Neural Networks    4
Conv1D             1
Name: count, dtype: int64

<IPython.core.display.Javascript object>

In [268]:
df_sorted_topo_models_grouped = df_sorted_topo_models_cols.groupby("Model").describe().T

<IPython.core.display.Javascript object>

In [269]:
# pd.set_option("display.max_rows", None)
# df_sorted_topo_models_grouped

<IPython.core.display.Javascript object>

In [270]:
df_sorted_topo_models_cols = df_sorted_topo[
    [
        "Model",
        "Dominated_Count",
        "Dominates_Count",
        "Classification",
    ]
]

<IPython.core.display.Javascript object>

In [271]:
(
    df_sorted_topo_models_cols.groupby("Model")[["Classification"]]
    .value_counts(normalize=True)
    .to_frame()
    * 100
).round(2)

proportion
Model           Classification                
Conv1D          intermediate_model       100.0
Neural Networks intermediate_model        50.0
                dominant_model            25.0
                non_dominant_model        25.0

<IPython.core.display.Javascript object>

In [272]:
df_sorted_topo_models_cols.groupby("Model").describe().round(2).T

Model                  Conv1D  Neural Networks
Dominated_Count count     1.0             4.00
                mean      1.0             1.50
                std       NaN             1.73
                min       1.0             0.00
                25%       1.0             0.75
                50%       1.0             1.00
                75%       1.0             1.75
                max       1.0             4.00
Dominates_Count count     1.0             4.00
                mean      1.0             1.50
                std       NaN             1.73
                min       1.0             0.00
                25%       1.0             0.75
                50%       1.0             1.00
                75%       1.0             1.75
                max       1.0             4.00

<IPython.core.display.Javascript object>

In [273]:
# df_sorted_topo_models_cols

<IPython.core.display.Javascript object>

In [274]:
df_sorted_topo_models_cols.groupby("Model")["Classification"].describe()

,count,unique,top,freq
Model,,,,
Conv1D,1,1,intermediate_model,1
Neural Networks,4,3,intermediate_model,2


<IPython.core.display.Javascript object>

In [275]:
df_sorted_topo_models_cols.groupby("Model")[["Classification"]].value_counts()

Model            Classification    
Conv1D           intermediate_model    1
Neural Networks  intermediate_model    2
                 dominant_model        1
                 non_dominant_model    1
Name: count, dtype: int64

<IPython.core.display.Javascript object>

In [276]:
# df_sorted_topo_models_cols

<IPython.core.display.Javascript object>

In [277]:
summary_stats = df_sorted_topo_models_cols.groupby("Model").agg(
    Total_Models=("Model", "count"),
    Mean_Dominance_Count=("Dominates_Count", "mean"),
    Median_Dominance_Count=("Dominates_Count", "median"),
    Total_Dominated=("Dominated_Count", "sum"),
    Total_Dominating=("Dominates_Count", "sum"),
)

total_models = summary_stats[
    "Total_Models"
].sum()  # Get the total number of models across all categories
summary_stats["Dominance Proportion"] = summary_stats["Total_Dominating"] / total_models

<IPython.core.display.Javascript object>

In [278]:
summary_stats

,Total_Models,Mean_Dominance_Count,Median_Dominance_Count,Total_Dominated,Total_Dominating,Dominance Proportion
Model,,,,,,
Conv1D,1,1.0,1.0,1,1,0.2
Neural Networks,4,1.5,1.0,6,6,1.2


<IPython.core.display.Javascript object>

In [279]:
df_sorted_topo_models_grouped.T

RMSE_mean                                                  \
                    count      mean      std      min       25%       50%   
Model                                                                       
Conv1D                1.0  1.420810      NaN  1.42081  1.420810  1.420810   
Neural Networks       4.0  1.756079  0.59219  1.37506  1.468697  1.504833   

                                    RMSE_std            ... Dominated_Count  \
                      75%       max    count      mean  ...             75%   
Model                                                   ...                   
Conv1D           1.420810  1.420810      1.0  0.111836  ...            1.00   
Neural Networks  1.792215  2.639592      4.0  0.662328  ...            1.75   

                     Dominates_Count                                            
                 max           count mean       std  min   25%  50%   75%  max  
Model                                                                           
Conv1D           1.0             1.0  1.0       NaN  1.0  1.00  1.0  1.00  1.0  
Neural Networks  4.0             4.0  1.5  1.732051  0.0  0.75  1.0  1.75  4.0  

[2 rows x 88 columns]

<IPython.core.display.Javascript object>

### SCPM Analysis

In [280]:
cols = [
    # "Model",
    # "Model_bkp",
    "Model_bkp_2",
    "RMSE_mean",
    "RMSE_std",
    "MAE_mean",
    "MAE_std",
    "MAPE_mean",
    "MAPE_std",
    "R2_mean",
    "R2_std",
    "SCPM",
    "Dominated_Count",
    "Dominates_Count",
    "Classification",
    "Plant",
]

<IPython.core.display.Javascript object>

#### SCPM Analysis by Project

In [281]:
df_sorted_scpm = df_sorted_topo.sort_values(by="SCPM").copy()

<IPython.core.display.Javascript object>

In [282]:
# (df_sorted_topo[cols].reset_index() == df_sorted_scpm[cols].reset_index()).all(axis=1)
df_sorted_scpm["topo_scmp_order_eq"] = df_sorted_topo.index == df_sorted_scpm.index

<IPython.core.display.Javascript object>

In [283]:
df_sorted_scpm[cols + ["topo_scmp_order_eq"]].round(2).head(5)

,Model_bkp_2,RMSE_mean,RMSE_std,MAE_mean,MAE_std,MAPE_mean,MAPE_std,R2_mean,R2_std,SCPM,Dominated_Count,Dominates_Count,Classification,Plant,topo_scmp_order_eq
4,Transformer_1.0,1.38,0.30,0.87,0.15,2.09,0.29,0.94,0.02,-3.27,0,4,dominant_model,c,True
2,LSTM_1.0,1.50,0.30,1.06,0.10,2.55,0.24,0.93,0.03,-1.60,1,1,intermediate_model,c,False
3,Conv1D_7.0,1.42,0.11,1.12,0.07,2.67,0.17,0.94,0.01,-1.49,1,1,intermediate_model,c,False
0,MLP,1.51,0.25,1.07,0.13,2.62,0.27,0.93,0.02,-1.47,1,1,intermediate_model,c,False
1,BidirectionalLSTM_1.0,2.64,1.81,1.74,1.10,4.18,2.71,0.71,0.40,7.82,4,0,non_dominant_model,c,True


<IPython.core.display.Javascript object>

### Time Series Split

In [284]:
df_results_cm_p = (
    df_results[
        df_results["Features"].eq("Chemical + Mineralogical + Physical - Early CS")
    ]
    .copy()
    .reset_index(drop=True)
)

<IPython.core.display.Javascript object>

In [285]:
df_results_cm_p["Features"].unique()

array(['Chemical + Mineralogical + Physical - Early CS'], dtype=object)

<IPython.core.display.Javascript object>

In [286]:
df_results_cm_p.shape

(10, 17)

<IPython.core.display.Javascript object>

### Plant C

In [287]:
df_results_cm_p = df_results_cm_p[df_results_cm_p["Plant"].eq("c")].reset_index(
    drop=True
)

<IPython.core.display.Javascript object>

In [288]:
df_results_cm["Cross Validation"].unique()

array(['Blocking Time Series Split', 'Time Series Split'], dtype=object)

<IPython.core.display.Javascript object>

In [289]:
df_results_cm_p_tss = (
    df_results_cm_p[df_results_cm_p["Cross Validation"].eq("Time Series Split")]
    .copy()
    .reset_index(drop=True)
)

<IPython.core.display.Javascript object>

In [290]:
df_results_cm_p_tss["Cross Validation"].unique()

array(['Time Series Split'], dtype=object)

<IPython.core.display.Javascript object>

In [291]:
df_results_cm_p_tss.shape

(5, 17)

<IPython.core.display.Javascript object>

###### SCPM computation

In [292]:
df_results_cm_p_tss = compute_scpm(df_results_cm_p_tss)

<IPython.core.display.Javascript object>

##### Dominance Analysis

In [293]:
# Per project
dominance_dict = make_dominance_analysis(df_results_cm_p_tss)
dominance_matrix_cm_p_tss = dominance_dict["dominance_matrix"]
dominance_graph_cm_p_tss = dominance_dict["dominance_graph"]
df_sorted_count = dominance_dict["df_sorted_count"]
df_sorted_topo = dominance_dict["df_sorted_topo"]

<IPython.core.display.Javascript object>

In [294]:
dominance_matrix_cm_tss.shape, len(dominance_graph_cm_tss)

((5, 5), 5)

<IPython.core.display.Javascript object>

In [295]:
df_sorted_topo["Classification"].value_counts()

Classification
intermediate_model    3
dominant_model        1
non_dominant_model    1
Name: count, dtype: int64

<IPython.core.display.Javascript object>

In [296]:
df_sorted_topo.to_csv(
    "../../../../reports/results/global_models/ecics/ecics_203_c_dominance_analysis_cm_p_tss.csv",
    index=False,
)

<IPython.core.display.Javascript object>

In [297]:
df_sorted_topo.head(5)

,Company,Plant,Cross Validation,Features,Features_bkp,Model,Model_bkp,Model_bkp_2,Timesteps,RMSE_mean,...,MAPE_mean,R2_mean,RMSE_std,MAE_std,MAPE_std,R2_std,SCPM,Dominated_Count,Classification,Dominates_Count
4,203,c,Time Series Split,Chemical + Mineralogical + Physical - Early CS,Chemical + Properties CS Less,Neural Networks,Transformer,Transformer_1.0,1.0,1.996887,...,3.150278,0.881287,0.400775,0.229285,0.520561,0.043862,-4.564028,0,dominant_model,4
0,203,c,Time Series Split,Chemical + Mineralogical + Physical - Early CS,Chemical + Properties CS Less,Neural Networks,MLP,MLP,NaN,2.115961,...,3.740009,0.866250,0.474516,0.368959,0.704568,0.051766,-2.116037,1,intermediate_model,3
2,203,c,Time Series Split,Chemical + Mineralogical + Physical - Early CS,Chemical + Properties CS Less,Neural Networks,LSTM,LSTM_1.0,1.0,3.045311,...,3.830104,0.686100,1.331369,0.589228,1.203727,0.274246,-0.496935,2,intermediate_model,1
3,203,c,Time Series Split,Chemical + Mineralogical + Physical - Early CS,Chemical + Properties CS Less,Conv1D,Conv1D,Conv1D_7.0,7.0,2.557865,...,4.267445,0.785160,0.819452,0.633268,1.343355,0.126450,0.439492,2,intermediate_model,1
1,203,c,Time Series Split,Chemical + Mineralogical + Physical - Early CS,Chemical + Properties CS Less,Neural Networks,BidirectionalLSTM,BidirectionalLSTM_1.0,1.0,5.702331,...,4.662165,-0.333849,3.531040,0.266670,0.697932,1.696471,6.737508,4,non_dominant_model,0


<IPython.core.display.Javascript object>

In [298]:
df_sorted_topo_cols = df_sorted_topo[
    [
        # "Model",
        # "Model_bkp",
        "Model_bkp_2",
        "RMSE_mean",
        "RMSE_std",
        "MAE_mean",
        "MAE_std",
        "MAPE_mean",
        "MAPE_std",
        "R2_mean",
        "R2_std",
        "SCPM",
        "Dominated_Count",
        "Classification",
        "Dominates_Count",
        "Plant",
    ]
]  # .round(2)

<IPython.core.display.Javascript object>

###### Dominant Models

In [299]:
# Top 4
df_sorted_topo_cols.iloc[0:2].sort_values(by="Dominates_Count", ascending=False).round(
    2
)

,Model_bkp_2,RMSE_mean,RMSE_std,MAE_mean,MAE_std,MAPE_mean,MAPE_std,R2_mean,R2_std,SCPM,Dominated_Count,Classification,Dominates_Count,Plant
4,Transformer_1.0,2.00,0.40,1.37,0.23,3.15,0.52,0.88,0.04,-4.56,0,dominant_model,4,c
0,MLP,2.12,0.47,1.62,0.37,3.74,0.70,0.87,0.05,-2.12,1,intermediate_model,3,c


<IPython.core.display.Javascript object>

In [300]:
df_sorted_topo_cols[df_sorted_topo_cols["Classification"].eq("dominant_model")].shape

(1, 14)

<IPython.core.display.Javascript object>

###### Intermediate models

In [301]:
df_sorted_topo_cols[
    df_sorted_topo_cols["Classification"].eq("intermediate_model")
].sort_values(by=["Dominates_Count", "SCPM"], ascending=[False, True]).iloc[:4].round(2)

,Model_bkp_2,RMSE_mean,RMSE_std,MAE_mean,MAE_std,MAPE_mean,MAPE_std,R2_mean,R2_std,SCPM,Dominated_Count,Classification,Dominates_Count,Plant
0,MLP,2.12,0.47,1.62,0.37,3.74,0.70,0.87,0.05,-2.12,1,intermediate_model,3,c
2,LSTM_1.0,3.05,1.33,1.70,0.59,3.83,1.20,0.69,0.27,-0.50,2,intermediate_model,1,c
3,Conv1D_7.0,2.56,0.82,1.85,0.63,4.27,1.34,0.79,0.13,0.44,2,intermediate_model,1,c


<IPython.core.display.Javascript object>

In [302]:
df_sorted_topo_cols[
    df_sorted_topo_cols["Classification"].eq("intermediate_model")
].shape

(3, 14)

<IPython.core.display.Javascript object>

###### non-dominant models

In [303]:
df_sorted_topo_cols[
    df_sorted_topo_cols["Classification"].eq("non_dominant_model")
].sort_values(by=["Dominates_Count", "SCPM"], ascending=[False, True]).iloc[:3]

,Model_bkp_2,RMSE_mean,RMSE_std,MAE_mean,MAE_std,MAPE_mean,MAPE_std,R2_mean,R2_std,SCPM,Dominated_Count,Classification,Dominates_Count,Plant
1,BidirectionalLSTM_1.0,5.702331,3.53104,2.016202,0.26667,4.662165,0.697932,-0.333849,1.696471,6.737508,4,non_dominant_model,0,c


<IPython.core.display.Javascript object>

In [304]:
df_sorted_topo_cols[
    df_sorted_topo_cols["Classification"].eq("non_dominant_model")
].shape

(1, 14)

<IPython.core.display.Javascript object>

###### Statistics per model type

In [305]:
df_sorted_topo_models_cols = df_sorted_topo[
    [
        "Model",
        # "Model_bkp",
        # "Model_bkp_2",
        "RMSE_mean",
        "RMSE_std",
        "MAE_mean",
        "MAE_std",
        "MAPE_mean",
        "MAPE_std",
        "R2_mean",
        "R2_std",
        "SCPM",
        "Dominated_Count",
        "Dominates_Count",
        "Classification",
        "Plant",
    ]
]  # .round(2)

<IPython.core.display.Javascript object>

In [306]:
df_sorted_topo["Model"].value_counts()

Model
Neural Networks    4
Conv1D             1
Name: count, dtype: int64

<IPython.core.display.Javascript object>

In [307]:
df_sorted_topo_models_grouped = df_sorted_topo_models_cols.groupby("Model").describe().T

<IPython.core.display.Javascript object>

In [308]:
df_sorted_topo_models_cols = df_sorted_topo[
    [
        "Model",
        "Dominated_Count",
        "Dominates_Count",
        "Classification",
    ]
]

<IPython.core.display.Javascript object>

In [309]:
(
    df_sorted_topo_models_cols.groupby("Model")[["Classification"]]
    .value_counts(normalize=True)
    .to_frame()
    * 100
).round(2)

proportion
Model           Classification                
Conv1D          intermediate_model       100.0
Neural Networks intermediate_model        50.0
                dominant_model            25.0
                non_dominant_model        25.0

<IPython.core.display.Javascript object>

In [310]:
df_sorted_topo_models_cols.groupby("Model").describe().round(2).T

Model                  Conv1D  Neural Networks
Dominated_Count count     1.0             4.00
                mean      2.0             1.75
                std       NaN             1.71
                min       2.0             0.00
                25%       2.0             0.75
                50%       2.0             1.50
                75%       2.0             2.50
                max       2.0             4.00
Dominates_Count count     1.0             4.00
                mean      1.0             2.00
                std       NaN             1.83
                min       1.0             0.00
                25%       1.0             0.75
                50%       1.0             2.00
                75%       1.0             3.25
                max       1.0             4.00

<IPython.core.display.Javascript object>

In [311]:
# df_sorted_topo_models_cols

<IPython.core.display.Javascript object>

In [312]:
df_sorted_topo_models_cols.groupby("Model")["Classification"].describe()

,count,unique,top,freq
Model,,,,
Conv1D,1,1,intermediate_model,1
Neural Networks,4,3,intermediate_model,2


<IPython.core.display.Javascript object>

In [313]:
df_sorted_topo_models_cols.groupby("Model")[["Classification"]].value_counts()

Model            Classification    
Conv1D           intermediate_model    1
Neural Networks  intermediate_model    2
                 dominant_model        1
                 non_dominant_model    1
Name: count, dtype: int64

<IPython.core.display.Javascript object>

In [314]:
summary_stats = df_sorted_topo_models_cols.groupby("Model").agg(
    Total_Models=("Model", "count"),
    Mean_Dominance_Count=("Dominates_Count", "mean"),
    Median_Dominance_Count=("Dominates_Count", "median"),
    Total_Dominated=("Dominated_Count", "sum"),
    Total_Dominating=("Dominates_Count", "sum"),
)

total_models = summary_stats[
    "Total_Models"
].sum()  # Get the total number of models across all categories
summary_stats["Dominance Proportion"] = summary_stats["Total_Dominating"] / total_models

<IPython.core.display.Javascript object>

In [315]:
summary_stats.round(2)

,Total_Models,Mean_Dominance_Count,Median_Dominance_Count,Total_Dominated,Total_Dominating,Dominance Proportion
Model,,,,,,
Conv1D,1,1.0,1.0,2,1,0.2
Neural Networks,4,2.0,2.0,7,8,1.6


<IPython.core.display.Javascript object>

In [316]:
df_sorted_topo_models_grouped.T

RMSE_mean                                                    \
                    count      mean       std       min       25%       50%   
Model                                                                         
Conv1D                1.0  2.557865       NaN  2.557865  2.557865  2.557865   
Neural Networks       4.0  3.215122  1.723107  1.996887  2.086193  2.580636   

                                    RMSE_std            ... Dominated_Count  \
                      75%       max    count      mean  ...             75%   
Model                                                   ...                   
Conv1D           2.557865  2.557865      1.0  0.819452  ...             2.0   
Neural Networks  3.709566  5.702331      4.0  1.434425  ...             2.5   

                     Dominates_Count                                            
                 max           count mean       std  min   25%  50%   75%  max  
Model                                                                           
Conv1D           2.0             1.0  1.0       NaN  1.0  1.00  1.0  1.00  1.0  
Neural Networks  4.0             4.0  2.0  1.825742  0.0  0.75  2.0  3.25  4.0  

[2 rows x 88 columns]

<IPython.core.display.Javascript object>

### SCPM Analysis

In [317]:
cols = [
    # "Model",
    # "Model_bkp",
    "Model_bkp_2",
    "RMSE_mean",
    "RMSE_std",
    "MAE_mean",
    "MAE_std",
    "MAPE_mean",
    "MAPE_std",
    "R2_mean",
    "R2_std",
    "SCPM",
    "Dominated_Count",
    "Dominates_Count",
    "Classification",
    "Plant",
]

<IPython.core.display.Javascript object>

#### SCPM Analysis by Project

In [318]:
df_sorted_scpm = df_sorted_topo.sort_values(by="SCPM").copy()

<IPython.core.display.Javascript object>

In [319]:
# (df_sorted_topo[cols].reset_index() == df_sorted_scpm[cols].reset_index()).all(axis=1)
df_sorted_scpm["topo_scmp_order_eq"] = df_sorted_topo.index == df_sorted_scpm.index

<IPython.core.display.Javascript object>

In [320]:
df_sorted_scpm[cols + ["topo_scmp_order_eq"]].round(2).head(5)

,Model_bkp_2,RMSE_mean,RMSE_std,MAE_mean,MAE_std,MAPE_mean,MAPE_std,R2_mean,R2_std,SCPM,Dominated_Count,Dominates_Count,Classification,Plant,topo_scmp_order_eq
4,Transformer_1.0,2.00,0.40,1.37,0.23,3.15,0.52,0.88,0.04,-4.56,0,4,dominant_model,c,True
0,MLP,2.12,0.47,1.62,0.37,3.74,0.70,0.87,0.05,-2.12,1,3,intermediate_model,c,True
2,LSTM_1.0,3.05,1.33,1.70,0.59,3.83,1.20,0.69,0.27,-0.50,2,1,intermediate_model,c,True
3,Conv1D_7.0,2.56,0.82,1.85,0.63,4.27,1.34,0.79,0.13,0.44,2,1,intermediate_model,c,True
1,BidirectionalLSTM_1.0,5.70,3.53,2.02,0.27,4.66,0.70,-0.33,1.70,6.74,4,0,non_dominant_model,c,True


<IPython.core.display.Javascript object>

## Variable Grouping: CM-P-CS

#### Analysis of the best models identified in the Local Analysis

**Best models for variable groupping CM-P-CS were from:**

*BTiSS:*
    Dominance analysis: Plant S and Plant K
    SCPM: Plant S and Plant K

*TiSS:*
    Dominance analysis: Plant S
    SCPM: Plant S

In [321]:
df_results["Features"].unique()

array(['Chemical + Mineralogical', 'Chemical + Mineralogical + Physical',
       'Chemical + Mineralogical + Physical - Early CS'], dtype=object)

<IPython.core.display.Javascript object>

In [322]:
df_results_cm_p_cs = (
    df_results[df_results["Features"].eq("Chemical + Mineralogical + Physical")]
    .copy()
    .reset_index(drop=True)
)

<IPython.core.display.Javascript object>

In [323]:
df_results_cm_p_cs["Features"].unique()

array(['Chemical + Mineralogical + Physical'], dtype=object)

<IPython.core.display.Javascript object>

In [324]:
df_results_cm_p_cs.shape

(10, 17)

<IPython.core.display.Javascript object>

### Plant C

In [325]:
df_results_cm_p_cs = df_results_cm_p_cs[
    df_results_cm_p_cs["Plant"].eq("c")
].reset_index(drop=True)

<IPython.core.display.Javascript object>

### Blocking time series

In [326]:
df_results_cm_p_cs["Cross Validation"].unique()

array(['Blocking Time Series Split', 'Time Series Split'], dtype=object)

<IPython.core.display.Javascript object>

In [327]:
df_results_cm_p_cs_btss = (
    df_results_cm_p_cs[
        df_results_cm_p_cs["Cross Validation"].eq("Blocking Time Series Split")
    ]
    .copy()
    .reset_index(drop=True)
)

<IPython.core.display.Javascript object>

In [328]:
df_results_cm_p_cs_btss["Cross Validation"].unique()

array(['Blocking Time Series Split'], dtype=object)

<IPython.core.display.Javascript object>

In [329]:
df_results_cm_p_cs_btss.shape

(5, 17)

<IPython.core.display.Javascript object>

###### SCPM computation

In [330]:
df_results_cm_p_cs_btss = compute_scpm(df_results_cm_p_cs_btss)

<IPython.core.display.Javascript object>

##### Dominance Analysis

In [331]:
# Per project
dominance_dict = make_dominance_analysis(df_results_cm_p_cs_btss)
dominance_matrix_cm_p_cs_btss = dominance_dict["dominance_matrix"]
dominance_graph_cm_p_cs_btss = dominance_dict["dominance_graph"]
df_sorted_count = dominance_dict["df_sorted_count"]
df_sorted_topo = dominance_dict["df_sorted_topo"]

<IPython.core.display.Javascript object>

In [332]:
dominance_matrix_cm_p_cs_btss.shape, len(dominance_graph_cm_p_cs_btss)

((5, 5), 5)

<IPython.core.display.Javascript object>

In [333]:
df_sorted_topo["Classification"].value_counts()

Classification
intermediate_model    3
dominant_model        1
non_dominant_model    1
Name: count, dtype: int64

<IPython.core.display.Javascript object>

In [334]:
df_sorted_topo.to_csv(
    "../../../../reports/results/global_models/ecics/ecics_203_c_dominance_analysis_cm_p_cs_btss.csv",
    index=False,
)

<IPython.core.display.Javascript object>

In [335]:
df_sorted_topo.head(5)

,Company,Plant,Cross Validation,Features,Features_bkp,Model,Model_bkp,Model_bkp_2,Timesteps,RMSE_mean,...,MAPE_mean,R2_mean,RMSE_std,MAE_std,MAPE_std,R2_std,SCPM,Dominated_Count,Classification,Dominates_Count
0,203,c,Blocking Time Series Split,Chemical + Mineralogical + Physical,Chemical + Physical,Neural Networks,MLP,MLP,NaN,1.083815,...,2.040679,0.966125,0.127806,0.089256,0.240282,0.005697,-3.068285,0,dominant_model,4
3,203,c,Blocking Time Series Split,Chemical + Mineralogical + Physical,Chemical + Physical,Neural Networks,LSTM,LSTM_1.0,1.0,1.254072,...,2.295953,0.951814,0.339233,0.175088,0.465900,0.026387,-2.572658,1,intermediate_model,3
4,203,c,Blocking Time Series Split,Chemical + Mineralogical + Physical,Chemical + Physical,Neural Networks,BidirectionalLSTM,BidirectionalLSTM_1.0,1.0,1.653175,...,2.821367,0.915692,0.499639,0.256269,0.815318,0.049126,-1.550182,2,intermediate_model,2
2,203,c,Blocking Time Series Split,Chemical + Mineralogical + Physical,Chemical + Physical,Neural Networks,Transformer,Transformer_1.0,1.0,2.028573,...,3.159316,0.831876,1.397818,0.616126,1.998640,0.229731,-0.590366,3,intermediate_model,1
1,203,c,Blocking Time Series Split,Chemical + Mineralogical + Physical,Chemical + Physical,Conv1D,Conv1D,Conv1D_14.0,14.0,3.644053,...,7.659026,0.202767,3.828968,3.668491,8.914510,1.391619,7.781491,4,non_dominant_model,0


<IPython.core.display.Javascript object>

In [336]:
df_sorted_topo_cols = df_sorted_topo[
    [
        # "Model",
        # "Model_bkp",
        "Model_bkp_2",
        "RMSE_mean",
        "RMSE_std",
        "MAE_mean",
        "MAE_std",
        "MAPE_mean",
        "MAPE_std",
        "R2_mean",
        "R2_std",
        "SCPM",
        "Dominated_Count",
        "Classification",
        "Dominates_Count",
        "Plant",
    ]
].round(2)

<IPython.core.display.Javascript object>

###### Dominant Models

In [337]:
# Top 4
df_sorted_topo_cols.iloc[0:5]

,Model_bkp_2,RMSE_mean,RMSE_std,MAE_mean,MAE_std,MAPE_mean,MAPE_std,R2_mean,R2_std,SCPM,Dominated_Count,Classification,Dominates_Count,Plant
0,MLP,1.08,0.13,0.84,0.09,2.04,0.24,0.97,0.01,-3.07,0,dominant_model,4,c
3,LSTM_1.0,1.25,0.34,0.96,0.18,2.30,0.47,0.95,0.03,-2.57,1,intermediate_model,3,c
4,BidirectionalLSTM_1.0,1.65,0.50,1.14,0.26,2.82,0.82,0.92,0.05,-1.55,2,intermediate_model,2,c
2,Transformer_1.0,2.03,1.40,1.23,0.62,3.16,2.00,0.83,0.23,-0.59,3,intermediate_model,1,c
1,Conv1D_14.0,3.64,3.83,3.21,3.67,7.66,8.91,0.20,1.39,7.78,4,non_dominant_model,0,c


<IPython.core.display.Javascript object>

In [338]:
df_sorted_topo_cols[df_sorted_topo_cols["Classification"].eq("dominant_model")].shape

(1, 14)

<IPython.core.display.Javascript object>

###### Intermediate models

In [339]:
df_sorted_topo_cols[
    df_sorted_topo_cols["Classification"].eq("intermediate_model")
].sort_values(by=["Dominates_Count", "SCPM"], ascending=[False, True]).iloc[:3]

,Model_bkp_2,RMSE_mean,RMSE_std,MAE_mean,MAE_std,MAPE_mean,MAPE_std,R2_mean,R2_std,SCPM,Dominated_Count,Classification,Dominates_Count,Plant
3,LSTM_1.0,1.25,0.34,0.96,0.18,2.30,0.47,0.95,0.03,-2.57,1,intermediate_model,3,c
4,BidirectionalLSTM_1.0,1.65,0.50,1.14,0.26,2.82,0.82,0.92,0.05,-1.55,2,intermediate_model,2,c
2,Transformer_1.0,2.03,1.40,1.23,0.62,3.16,2.00,0.83,0.23,-0.59,3,intermediate_model,1,c


<IPython.core.display.Javascript object>

In [340]:
df_sorted_topo_cols[
    df_sorted_topo_cols["Classification"].eq("intermediate_model")
].shape

(3, 14)

<IPython.core.display.Javascript object>

###### non-dominant models

In [341]:
df_sorted_topo_cols[
    df_sorted_topo_cols["Classification"].eq("non_dominant_model")
].sort_values(by=["Dominates_Count", "SCPM"], ascending=[False, True]).iloc[:2]

,Model_bkp_2,RMSE_mean,RMSE_std,MAE_mean,MAE_std,MAPE_mean,MAPE_std,R2_mean,R2_std,SCPM,Dominated_Count,Classification,Dominates_Count,Plant
1,Conv1D_14.0,3.64,3.83,3.21,3.67,7.66,8.91,0.2,1.39,7.78,4,non_dominant_model,0,c


<IPython.core.display.Javascript object>

In [342]:
df_sorted_topo_cols[
    df_sorted_topo_cols["Classification"].eq("non_dominant_model")
].shape

(1, 14)

<IPython.core.display.Javascript object>

###### Statistics per model type

In [343]:
df_sorted_topo_models_cols = df_sorted_topo[
    [
        "Model",
        # "Model_bkp",
        # "Model_bkp_2",
        "RMSE_mean",
        "RMSE_std",
        "MAE_mean",
        "MAE_std",
        "MAPE_mean",
        "MAPE_std",
        "R2_mean",
        "R2_std",
        "SCPM",
        "Dominated_Count",
        "Dominates_Count",
        "Classification",
        "Plant",
    ]
]  # .round(2)

<IPython.core.display.Javascript object>

In [344]:
df_sorted_topo["Model"].value_counts()

Model
Neural Networks    4
Conv1D             1
Name: count, dtype: int64

<IPython.core.display.Javascript object>

In [345]:
df_sorted_topo_models_grouped = df_sorted_topo_models_cols.groupby("Model").describe().T

<IPython.core.display.Javascript object>

In [346]:
# pd.set_option("display.max_rows", None)
# df_sorted_topo_models_grouped

<IPython.core.display.Javascript object>

In [347]:
df_sorted_topo_models_cols = df_sorted_topo[
    [
        "Model",
        "Dominated_Count",
        "Dominates_Count",
        "Classification",
    ]
]

<IPython.core.display.Javascript object>

In [348]:
(
    df_sorted_topo_models_cols.groupby("Model")[["Classification"]]
    .value_counts(normalize=True)
    .to_frame()
    * 100
).round(2)

proportion
Model           Classification                
Conv1D          non_dominant_model       100.0
Neural Networks intermediate_model        75.0
                dominant_model            25.0

<IPython.core.display.Javascript object>

In [349]:
df_sorted_topo_models_cols.groupby("Model").describe().round(2).T

Model                  Conv1D  Neural Networks
Dominated_Count count     1.0             4.00
                mean      4.0             1.50
                std       NaN             1.29
                min       4.0             0.00
                25%       4.0             0.75
                50%       4.0             1.50
                75%       4.0             2.25
                max       4.0             3.00
Dominates_Count count     1.0             4.00
                mean      0.0             2.50
                std       NaN             1.29
                min       0.0             1.00
                25%       0.0             1.75
                50%       0.0             2.50
                75%       0.0             3.25
                max       0.0             4.00

<IPython.core.display.Javascript object>

In [350]:
# df_sorted_topo_models_cols

<IPython.core.display.Javascript object>

In [351]:
df_sorted_topo_models_cols.groupby("Model")["Classification"].describe()

,count,unique,top,freq
Model,,,,
Conv1D,1,1,non_dominant_model,1
Neural Networks,4,2,intermediate_model,3


<IPython.core.display.Javascript object>

In [352]:
df_sorted_topo_models_cols.groupby("Model")[["Classification"]].value_counts()

Model            Classification    
Conv1D           non_dominant_model    1
Neural Networks  intermediate_model    3
                 dominant_model        1
Name: count, dtype: int64

<IPython.core.display.Javascript object>

In [353]:
# df_sorted_topo_models_cols

<IPython.core.display.Javascript object>

In [354]:
summary_stats = df_sorted_topo_models_cols.groupby("Model").agg(
    Total_Models=("Model", "count"),
    Mean_Dominance_Count=("Dominates_Count", "mean"),
    Median_Dominance_Count=("Dominates_Count", "median"),
    Total_Dominated=("Dominated_Count", "sum"),
    Total_Dominating=("Dominates_Count", "sum"),
)

total_models = summary_stats[
    "Total_Models"
].sum()  # Get the total number of models across all categories
summary_stats["Dominance Proportion"] = summary_stats["Total_Dominating"] / total_models

<IPython.core.display.Javascript object>

In [355]:
summary_stats

,Total_Models,Mean_Dominance_Count,Median_Dominance_Count,Total_Dominated,Total_Dominating,Dominance Proportion
Model,,,,,,
Conv1D,1,0.0,0.0,4,0,0.0
Neural Networks,4,2.5,2.5,6,10,2.0


<IPython.core.display.Javascript object>

In [356]:
df_sorted_topo_models_grouped.T

RMSE_mean                                                    \
                    count      mean       std       min       25%       50%   
Model                                                                         
Conv1D                1.0  3.644053       NaN  3.644053  3.644053  3.644053   
Neural Networks       4.0  1.504909  0.422866  1.083815  1.211507  1.453623   

                                    RMSE_std            ... Dominated_Count  \
                      75%       max    count      mean  ...             75%   
Model                                                   ...                   
Conv1D           3.644053  3.644053      1.0  3.828968  ...            4.00   
Neural Networks  1.747024  2.028573      4.0  0.591124  ...            2.25   

                     Dominates_Count                                            
                 max           count mean       std  min   25%  50%   75%  max  
Model                                                                           
Conv1D           4.0             1.0  0.0       NaN  0.0  0.00  0.0  0.00  0.0  
Neural Networks  3.0             4.0  2.5  1.290994  1.0  1.75  2.5  3.25  4.0  

[2 rows x 88 columns]

<IPython.core.display.Javascript object>

### SCPM Analysis

In [357]:
cols = [
    # "Model",
    # "Model_bkp",
    "Model_bkp_2",
    "RMSE_mean",
    "RMSE_std",
    "MAE_mean",
    "MAE_std",
    "MAPE_mean",
    "MAPE_std",
    "R2_mean",
    "R2_std",
    "SCPM",
    "Dominated_Count",
    "Dominates_Count",
    "Classification",
    "Plant",
]

<IPython.core.display.Javascript object>

#### SCPM Analysis by Project

In [358]:
df_sorted_scpm = df_sorted_topo.sort_values(by="SCPM").copy()

<IPython.core.display.Javascript object>

In [359]:
# (df_sorted_topo[cols].reset_index() == df_sorted_scpm[cols].reset_index()).all(axis=1)
df_sorted_scpm["topo_scmp_order_eq"] = df_sorted_topo.index == df_sorted_scpm.index

<IPython.core.display.Javascript object>

In [360]:
df_sorted_scpm[cols + ["topo_scmp_order_eq"]].round(2).head(5)

,Model_bkp_2,RMSE_mean,RMSE_std,MAE_mean,MAE_std,MAPE_mean,MAPE_std,R2_mean,R2_std,SCPM,Dominated_Count,Dominates_Count,Classification,Plant,topo_scmp_order_eq
0,MLP,1.08,0.13,0.84,0.09,2.04,0.24,0.97,0.01,-3.07,0,4,dominant_model,c,True
3,LSTM_1.0,1.25,0.34,0.96,0.18,2.30,0.47,0.95,0.03,-2.57,1,3,intermediate_model,c,True
4,BidirectionalLSTM_1.0,1.65,0.50,1.14,0.26,2.82,0.82,0.92,0.05,-1.55,2,2,intermediate_model,c,True
2,Transformer_1.0,2.03,1.40,1.23,0.62,3.16,2.00,0.83,0.23,-0.59,3,1,intermediate_model,c,True
1,Conv1D_14.0,3.64,3.83,3.21,3.67,7.66,8.91,0.20,1.39,7.78,4,0,non_dominant_model,c,True


<IPython.core.display.Javascript object>

### Plant C

In [361]:
df_results_cm_p_cs = (
    df_results[df_results["Features"].eq("Chemical + Mineralogical + Physical")]
    .copy()
    .reset_index(drop=True)
)

<IPython.core.display.Javascript object>

In [362]:
df_results_cm_p_cs["Features"].unique()

array(['Chemical + Mineralogical + Physical'], dtype=object)

<IPython.core.display.Javascript object>

In [363]:
df_results_cm_p_cs.shape

(10, 17)

<IPython.core.display.Javascript object>

In [364]:
df_results_cm_p_cs = df_results_cm_p_cs[
    df_results_cm_p_cs["Plant"].eq("c")
].reset_index(drop=True)

<IPython.core.display.Javascript object>

### Blocking time series

In [365]:
df_results_cm_p_cs["Cross Validation"].unique()

array(['Blocking Time Series Split', 'Time Series Split'], dtype=object)

<IPython.core.display.Javascript object>

In [366]:
df_results_cm_p_cs_btss = (
    df_results_cm_p_cs[
        df_results_cm_p_cs["Cross Validation"].eq("Blocking Time Series Split")
    ]
    .copy()
    .reset_index(drop=True)
)

<IPython.core.display.Javascript object>

In [367]:
df_results_cm_p_cs_btss["Cross Validation"].unique()

array(['Blocking Time Series Split'], dtype=object)

<IPython.core.display.Javascript object>

In [368]:
df_results_cm_p_cs_btss.shape

(5, 17)

<IPython.core.display.Javascript object>

###### SCPM computation

In [369]:
df_results_cm_p_cs_btss = compute_scpm(df_results_cm_p_cs_btss)

<IPython.core.display.Javascript object>

##### Dominance Analysis

In [370]:
# Per project
dominance_dict = make_dominance_analysis(df_results_cm_p_cs_btss)
dominance_matrix_cm_p_cs_btss = dominance_dict["dominance_matrix"]
dominance_graph_cm_p_cs_btss = dominance_dict["dominance_graph"]
df_sorted_count = dominance_dict["df_sorted_count"]
df_sorted_topo = dominance_dict["df_sorted_topo"]

<IPython.core.display.Javascript object>

In [371]:
dominance_matrix_cm_p_cs_btss.shape, len(dominance_graph_cm_p_cs_btss)

((5, 5), 5)

<IPython.core.display.Javascript object>

In [372]:
df_sorted_topo["Classification"].value_counts()

Classification
intermediate_model    3
dominant_model        1
non_dominant_model    1
Name: count, dtype: int64

<IPython.core.display.Javascript object>

In [373]:
df_sorted_topo.to_csv(
    "../../../../reports/results/global_models/ecics/ecics_209_k_dominance_analysis_cm_p_cs_btss.csv",
    index=False,
)

<IPython.core.display.Javascript object>

In [374]:
df_sorted_topo.head(5)

,Company,Plant,Cross Validation,Features,Features_bkp,Model,Model_bkp,Model_bkp_2,Timesteps,RMSE_mean,...,MAPE_mean,R2_mean,RMSE_std,MAE_std,MAPE_std,R2_std,SCPM,Dominated_Count,Classification,Dominates_Count
0,203,c,Blocking Time Series Split,Chemical + Mineralogical + Physical,Chemical + Physical,Neural Networks,MLP,MLP,NaN,1.083815,...,2.040679,0.966125,0.127806,0.089256,0.240282,0.005697,-3.068285,0,dominant_model,4
3,203,c,Blocking Time Series Split,Chemical + Mineralogical + Physical,Chemical + Physical,Neural Networks,LSTM,LSTM_1.0,1.0,1.254072,...,2.295953,0.951814,0.339233,0.175088,0.465900,0.026387,-2.572658,1,intermediate_model,3
4,203,c,Blocking Time Series Split,Chemical + Mineralogical + Physical,Chemical + Physical,Neural Networks,BidirectionalLSTM,BidirectionalLSTM_1.0,1.0,1.653175,...,2.821367,0.915692,0.499639,0.256269,0.815318,0.049126,-1.550182,2,intermediate_model,2
2,203,c,Blocking Time Series Split,Chemical + Mineralogical + Physical,Chemical + Physical,Neural Networks,Transformer,Transformer_1.0,1.0,2.028573,...,3.159316,0.831876,1.397818,0.616126,1.998640,0.229731,-0.590366,3,intermediate_model,1
1,203,c,Blocking Time Series Split,Chemical + Mineralogical + Physical,Chemical + Physical,Conv1D,Conv1D,Conv1D_14.0,14.0,3.644053,...,7.659026,0.202767,3.828968,3.668491,8.914510,1.391619,7.781491,4,non_dominant_model,0


<IPython.core.display.Javascript object>

In [375]:
df_sorted_topo_cols = df_sorted_topo[
    [
        # "Model",
        # "Model_bkp",
        "Model_bkp_2",
        "RMSE_mean",
        "RMSE_std",
        "MAE_mean",
        "MAE_std",
        "MAPE_mean",
        "MAPE_std",
        "R2_mean",
        "R2_std",
        "SCPM",
        "Dominated_Count",
        "Classification",
        "Dominates_Count",
        "Plant",
    ]
].round(2)

<IPython.core.display.Javascript object>

###### Dominant Models

In [376]:
# Top 4
df_sorted_topo_cols.iloc[0:5]

,Model_bkp_2,RMSE_mean,RMSE_std,MAE_mean,MAE_std,MAPE_mean,MAPE_std,R2_mean,R2_std,SCPM,Dominated_Count,Classification,Dominates_Count,Plant
0,MLP,1.08,0.13,0.84,0.09,2.04,0.24,0.97,0.01,-3.07,0,dominant_model,4,c
3,LSTM_1.0,1.25,0.34,0.96,0.18,2.30,0.47,0.95,0.03,-2.57,1,intermediate_model,3,c
4,BidirectionalLSTM_1.0,1.65,0.50,1.14,0.26,2.82,0.82,0.92,0.05,-1.55,2,intermediate_model,2,c
2,Transformer_1.0,2.03,1.40,1.23,0.62,3.16,2.00,0.83,0.23,-0.59,3,intermediate_model,1,c
1,Conv1D_14.0,3.64,3.83,3.21,3.67,7.66,8.91,0.20,1.39,7.78,4,non_dominant_model,0,c


<IPython.core.display.Javascript object>

In [377]:
df_sorted_topo_cols[df_sorted_topo_cols["Classification"].eq("dominant_model")].shape

(1, 14)

<IPython.core.display.Javascript object>

###### Intermediate models

In [378]:
df_sorted_topo_cols[
    df_sorted_topo_cols["Classification"].eq("intermediate_model")
].sort_values(by=["Dominates_Count", "SCPM"], ascending=[False, True]).iloc[:3]

,Model_bkp_2,RMSE_mean,RMSE_std,MAE_mean,MAE_std,MAPE_mean,MAPE_std,R2_mean,R2_std,SCPM,Dominated_Count,Classification,Dominates_Count,Plant
3,LSTM_1.0,1.25,0.34,0.96,0.18,2.30,0.47,0.95,0.03,-2.57,1,intermediate_model,3,c
4,BidirectionalLSTM_1.0,1.65,0.50,1.14,0.26,2.82,0.82,0.92,0.05,-1.55,2,intermediate_model,2,c
2,Transformer_1.0,2.03,1.40,1.23,0.62,3.16,2.00,0.83,0.23,-0.59,3,intermediate_model,1,c


<IPython.core.display.Javascript object>

In [379]:
df_sorted_topo_cols[
    df_sorted_topo_cols["Classification"].eq("intermediate_model")
].shape

(3, 14)

<IPython.core.display.Javascript object>

###### non-dominant models

In [380]:
df_sorted_topo_cols[
    df_sorted_topo_cols["Classification"].eq("non_dominant_model")
].sort_values(by=["Dominates_Count", "SCPM"], ascending=[False, True]).iloc[:2]

,Model_bkp_2,RMSE_mean,RMSE_std,MAE_mean,MAE_std,MAPE_mean,MAPE_std,R2_mean,R2_std,SCPM,Dominated_Count,Classification,Dominates_Count,Plant
1,Conv1D_14.0,3.64,3.83,3.21,3.67,7.66,8.91,0.2,1.39,7.78,4,non_dominant_model,0,c


<IPython.core.display.Javascript object>

In [381]:
df_sorted_topo_cols[
    df_sorted_topo_cols["Classification"].eq("non_dominant_model")
].shape

(1, 14)

<IPython.core.display.Javascript object>

###### Statistics per model type

In [382]:
df_sorted_topo_models_cols = df_sorted_topo[
    [
        "Model",
        # "Model_bkp",
        # "Model_bkp_2",
        "RMSE_mean",
        "RMSE_std",
        "MAE_mean",
        "MAE_std",
        "MAPE_mean",
        "MAPE_std",
        "R2_mean",
        "R2_std",
        "SCPM",
        "Dominated_Count",
        "Dominates_Count",
        "Classification",
        "Plant",
    ]
]  # .round(2)

<IPython.core.display.Javascript object>

In [383]:
df_sorted_topo["Model"].value_counts()

Model
Neural Networks    4
Conv1D             1
Name: count, dtype: int64

<IPython.core.display.Javascript object>

In [384]:
df_sorted_topo_models_grouped = df_sorted_topo_models_cols.groupby("Model").describe().T

<IPython.core.display.Javascript object>

In [385]:
# pd.set_option("display.max_rows", None)
# df_sorted_topo_models_grouped

<IPython.core.display.Javascript object>

In [386]:
df_sorted_topo_models_cols = df_sorted_topo[
    [
        "Model",
        "Dominated_Count",
        "Dominates_Count",
        "Classification",
    ]
]

<IPython.core.display.Javascript object>

In [387]:
(
    df_sorted_topo_models_cols.groupby("Model")[["Classification"]]
    .value_counts(normalize=True)
    .to_frame()
    * 100
).round(2)

proportion
Model           Classification                
Conv1D          non_dominant_model       100.0
Neural Networks intermediate_model        75.0
                dominant_model            25.0

<IPython.core.display.Javascript object>

In [388]:
df_sorted_topo_models_cols.groupby("Model").describe().round(2).T

Model                  Conv1D  Neural Networks
Dominated_Count count     1.0             4.00
                mean      4.0             1.50
                std       NaN             1.29
                min       4.0             0.00
                25%       4.0             0.75
                50%       4.0             1.50
                75%       4.0             2.25
                max       4.0             3.00
Dominates_Count count     1.0             4.00
                mean      0.0             2.50
                std       NaN             1.29
                min       0.0             1.00
                25%       0.0             1.75
                50%       0.0             2.50
                75%       0.0             3.25
                max       0.0             4.00

<IPython.core.display.Javascript object>

In [389]:
# df_sorted_topo_models_cols

<IPython.core.display.Javascript object>

In [390]:
df_sorted_topo_models_cols.groupby("Model")["Classification"].describe()

,count,unique,top,freq
Model,,,,
Conv1D,1,1,non_dominant_model,1
Neural Networks,4,2,intermediate_model,3


<IPython.core.display.Javascript object>

In [391]:
df_sorted_topo_models_cols.groupby("Model")[["Classification"]].value_counts()

Model            Classification    
Conv1D           non_dominant_model    1
Neural Networks  intermediate_model    3
                 dominant_model        1
Name: count, dtype: int64

<IPython.core.display.Javascript object>

In [392]:
# df_sorted_topo_models_cols

<IPython.core.display.Javascript object>

In [393]:
summary_stats = df_sorted_topo_models_cols.groupby("Model").agg(
    Total_Models=("Model", "count"),
    Mean_Dominance_Count=("Dominates_Count", "mean"),
    Median_Dominance_Count=("Dominates_Count", "median"),
    Total_Dominated=("Dominated_Count", "sum"),
    Total_Dominating=("Dominates_Count", "sum"),
)

total_models = summary_stats[
    "Total_Models"
].sum()  # Get the total number of models across all categories
summary_stats["Dominance Proportion"] = summary_stats["Total_Dominating"] / total_models

<IPython.core.display.Javascript object>

In [394]:
summary_stats

,Total_Models,Mean_Dominance_Count,Median_Dominance_Count,Total_Dominated,Total_Dominating,Dominance Proportion
Model,,,,,,
Conv1D,1,0.0,0.0,4,0,0.0
Neural Networks,4,2.5,2.5,6,10,2.0


<IPython.core.display.Javascript object>

In [395]:
df_sorted_topo_models_grouped.T

RMSE_mean                                                    \
                    count      mean       std       min       25%       50%   
Model                                                                         
Conv1D                1.0  3.644053       NaN  3.644053  3.644053  3.644053   
Neural Networks       4.0  1.504909  0.422866  1.083815  1.211507  1.453623   

                                    RMSE_std            ... Dominated_Count  \
                      75%       max    count      mean  ...             75%   
Model                                                   ...                   
Conv1D           3.644053  3.644053      1.0  3.828968  ...            4.00   
Neural Networks  1.747024  2.028573      4.0  0.591124  ...            2.25   

                     Dominates_Count                                            
                 max           count mean       std  min   25%  50%   75%  max  
Model                                                                           
Conv1D           4.0             1.0  0.0       NaN  0.0  0.00  0.0  0.00  0.0  
Neural Networks  3.0             4.0  2.5  1.290994  1.0  1.75  2.5  3.25  4.0  

[2 rows x 88 columns]

<IPython.core.display.Javascript object>

### SCPM Analysis

In [396]:
cols = [
    # "Model",
    # "Model_bkp",
    "Model_bkp_2",
    "RMSE_mean",
    "RMSE_std",
    "MAE_mean",
    "MAE_std",
    "MAPE_mean",
    "MAPE_std",
    "R2_mean",
    "R2_std",
    "SCPM",
    "Dominated_Count",
    "Dominates_Count",
    "Classification",
    "Plant",
]

<IPython.core.display.Javascript object>

#### SCPM Analysis by Project

In [397]:
df_sorted_scpm = df_sorted_topo.sort_values(by="SCPM").copy()

<IPython.core.display.Javascript object>

In [398]:
# (df_sorted_topo[cols].reset_index() == df_sorted_scpm[cols].reset_index()).all(axis=1)
df_sorted_scpm["topo_scmp_order_eq"] = df_sorted_topo.index == df_sorted_scpm.index

<IPython.core.display.Javascript object>

In [399]:
df_sorted_scpm[cols + ["topo_scmp_order_eq"]].round(2).head(5)

,Model_bkp_2,RMSE_mean,RMSE_std,MAE_mean,MAE_std,MAPE_mean,MAPE_std,R2_mean,R2_std,SCPM,Dominated_Count,Dominates_Count,Classification,Plant,topo_scmp_order_eq
0,MLP,1.08,0.13,0.84,0.09,2.04,0.24,0.97,0.01,-3.07,0,4,dominant_model,c,True
3,LSTM_1.0,1.25,0.34,0.96,0.18,2.30,0.47,0.95,0.03,-2.57,1,3,intermediate_model,c,True
4,BidirectionalLSTM_1.0,1.65,0.50,1.14,0.26,2.82,0.82,0.92,0.05,-1.55,2,2,intermediate_model,c,True
2,Transformer_1.0,2.03,1.40,1.23,0.62,3.16,2.00,0.83,0.23,-0.59,3,1,intermediate_model,c,True
1,Conv1D_14.0,3.64,3.83,3.21,3.67,7.66,8.91,0.20,1.39,7.78,4,0,non_dominant_model,c,True


<IPython.core.display.Javascript object>

### Time Series Split

In [400]:
df_results_cm_p_cs = (
    df_results[df_results["Features"].eq("Chemical + Mineralogical + Physical")]
    .copy()
    .reset_index(drop=True)
)

<IPython.core.display.Javascript object>

In [401]:
df_results_cm_p_cs["Features"].unique()

array(['Chemical + Mineralogical + Physical'], dtype=object)

<IPython.core.display.Javascript object>

In [402]:
df_results_cm_p_cs.shape

(10, 17)

<IPython.core.display.Javascript object>

### Plant C

In [403]:
df_results_cm_p_cs = df_results_cm_p_cs[
    df_results_cm_p_cs["Plant"].eq("c")
].reset_index(drop=True)

<IPython.core.display.Javascript object>

In [404]:
df_results_cm_p_cs["Cross Validation"].unique()

array(['Blocking Time Series Split', 'Time Series Split'], dtype=object)

<IPython.core.display.Javascript object>

In [405]:
df_results_cm_p_cs_tss = (
    df_results_cm_p_cs[df_results_cm_p_cs["Cross Validation"].eq("Time Series Split")]
    .copy()
    .reset_index(drop=True)
)

<IPython.core.display.Javascript object>

In [406]:
df_results_cm_p_cs_tss["Cross Validation"].unique()

array(['Time Series Split'], dtype=object)

<IPython.core.display.Javascript object>

In [407]:
df_results_cm_p_cs_tss.shape

(5, 17)

<IPython.core.display.Javascript object>

###### SCPM computation

In [408]:
df_results_cm_p_cs_tss = compute_scpm(df_results_cm_p_cs_tss)

<IPython.core.display.Javascript object>

##### Dominance Analysis

In [409]:
# Per project
dominance_dict = make_dominance_analysis(df_results_cm_p_cs_tss)
dominance_matrix_cm_p_cs_tss = dominance_dict["dominance_matrix"]
dominance_graph_cm_p_cs_tss = dominance_dict["dominance_graph"]
df_sorted_count = dominance_dict["df_sorted_count"]
df_sorted_topo = dominance_dict["df_sorted_topo"]

<IPython.core.display.Javascript object>

In [410]:
dominance_matrix_cm_p_cs_tss.shape, len(dominance_graph_cm_p_cs_tss)

((5, 5), 5)

<IPython.core.display.Javascript object>

In [411]:
df_sorted_topo["Classification"].value_counts()

Classification
intermediate_model    3
dominant_model        1
non_dominant_model    1
Name: count, dtype: int64

<IPython.core.display.Javascript object>

In [412]:
df_sorted_topo.to_csv(
    "../../../../reports/results/global_models/ecics/ecics_203_c_dominance_analysis_cm_p_cs_tss.csv",
    index=False,
)

<IPython.core.display.Javascript object>

In [413]:
df_sorted_topo.head(5)

,Company,Plant,Cross Validation,Features,Features_bkp,Model,Model_bkp,Model_bkp_2,Timesteps,RMSE_mean,...,MAPE_mean,R2_mean,RMSE_std,MAE_std,MAPE_std,R2_std,SCPM,Dominated_Count,Classification,Dominates_Count
0,203,c,Time Series Split,Chemical + Mineralogical + Physical,Chemical + Physical,Neural Networks,MLP,MLP,NaN,1.555784,...,2.545077,0.929168,0.217635,0.243941,0.472629,0.017320,-4.335186,0,dominant_model,4
1,203,c,Time Series Split,Chemical + Mineralogical + Physical,Chemical + Physical,Conv1D,Conv1D,Conv1D_14.0,14.0,1.887169,...,3.368246,0.890463,0.370187,0.263370,0.656618,0.045243,-1.604336,1,intermediate_model,2
3,203,c,Time Series Split,Chemical + Mineralogical + Physical,Chemical + Physical,Neural Networks,LSTM,LSTM_1.0,1.0,2.482138,...,3.266100,0.780624,1.204922,0.368196,0.732168,0.259648,-1.478699,1,intermediate_model,2
2,203,c,Time Series Split,Chemical + Mineralogical + Physical,Chemical + Physical,Neural Networks,Transformer,Transformer_1.0,1.0,2.653148,...,3.909742,0.732787,1.494346,0.906877,1.867841,0.322729,0.724551,3,intermediate_model,1
4,203,c,Time Series Split,Chemical + Mineralogical + Physical,Chemical + Physical,Neural Networks,BidirectionalLSTM,BidirectionalLSTM_1.0,1.0,10.232263,...,4.272492,-4.536506,8.699430,0.679207,1.617161,6.856794,6.693669,4,non_dominant_model,0


<IPython.core.display.Javascript object>

In [414]:
df_sorted_topo_cols = df_sorted_topo[
    [
        # "Model",
        # "Model_bkp",
        "Model_bkp_2",
        "RMSE_mean",
        "RMSE_std",
        "MAE_mean",
        "MAE_std",
        "MAPE_mean",
        "MAPE_std",
        "R2_mean",
        "R2_std",
        "SCPM",
        "Dominated_Count",
        "Classification",
        "Dominates_Count",
        "Plant",
    ]
]  # .round(2)

<IPython.core.display.Javascript object>

###### Dominant Models

In [415]:
# Top 4
df_sorted_topo_cols.iloc[0:2].sort_values(by="Dominates_Count", ascending=False).round(
    2
)

,Model_bkp_2,RMSE_mean,RMSE_std,MAE_mean,MAE_std,MAPE_mean,MAPE_std,R2_mean,R2_std,SCPM,Dominated_Count,Classification,Dominates_Count,Plant
0,MLP,1.56,0.22,1.10,0.24,2.55,0.47,0.93,0.02,-4.34,0,dominant_model,4,c
1,Conv1D_14.0,1.89,0.37,1.43,0.26,3.37,0.66,0.89,0.05,-1.60,1,intermediate_model,2,c


<IPython.core.display.Javascript object>

In [416]:
df_sorted_topo_cols[df_sorted_topo_cols["Classification"].eq("dominant_model")].shape

(1, 14)

<IPython.core.display.Javascript object>

###### Intermediate models

In [417]:
df_sorted_topo_cols[
    df_sorted_topo_cols["Classification"].eq("intermediate_model")
].sort_values(by=["Dominates_Count", "SCPM"], ascending=[False, True]).iloc[:4].round(2)

,Model_bkp_2,RMSE_mean,RMSE_std,MAE_mean,MAE_std,MAPE_mean,MAPE_std,R2_mean,R2_std,SCPM,Dominated_Count,Classification,Dominates_Count,Plant
1,Conv1D_14.0,1.89,0.37,1.43,0.26,3.37,0.66,0.89,0.05,-1.60,1,intermediate_model,2,c
3,LSTM_1.0,2.48,1.20,1.44,0.37,3.27,0.73,0.78,0.26,-1.48,1,intermediate_model,2,c
2,Transformer_1.0,2.65,1.49,1.72,0.91,3.91,1.87,0.73,0.32,0.72,3,intermediate_model,1,c


<IPython.core.display.Javascript object>

In [418]:
df_sorted_topo_cols[
    df_sorted_topo_cols["Classification"].eq("intermediate_model")
].shape

(3, 14)

<IPython.core.display.Javascript object>

###### non-dominant models

In [419]:
df_sorted_topo_cols[
    df_sorted_topo_cols["Classification"].eq("non_dominant_model")
].sort_values(by=["Dominates_Count", "SCPM"], ascending=[False, True]).iloc[:3]

,Model_bkp_2,RMSE_mean,RMSE_std,MAE_mean,MAE_std,MAPE_mean,MAPE_std,R2_mean,R2_std,SCPM,Dominated_Count,Classification,Dominates_Count,Plant
4,BidirectionalLSTM_1.0,10.232263,8.69943,1.875933,0.679207,4.272492,1.617161,-4.536506,6.856794,6.693669,4,non_dominant_model,0,c


<IPython.core.display.Javascript object>

In [420]:
df_sorted_topo_cols[
    df_sorted_topo_cols["Classification"].eq("non_dominant_model")
].shape

(1, 14)

<IPython.core.display.Javascript object>

###### Statistics per model type

In [421]:
df_sorted_topo_models_cols = df_sorted_topo[
    [
        "Model",
        # "Model_bkp",
        # "Model_bkp_2",
        "RMSE_mean",
        "RMSE_std",
        "MAE_mean",
        "MAE_std",
        "MAPE_mean",
        "MAPE_std",
        "R2_mean",
        "R2_std",
        "SCPM",
        "Dominated_Count",
        "Dominates_Count",
        "Classification",
        "Plant",
    ]
]  # .round(2)

<IPython.core.display.Javascript object>

In [422]:
df_sorted_topo["Model"].value_counts()

Model
Neural Networks    4
Conv1D             1
Name: count, dtype: int64

<IPython.core.display.Javascript object>

In [423]:
df_sorted_topo_models_grouped = df_sorted_topo_models_cols.groupby("Model").describe().T

<IPython.core.display.Javascript object>

In [424]:
df_sorted_topo_models_cols = df_sorted_topo[
    [
        "Model",
        "Dominated_Count",
        "Dominates_Count",
        "Classification",
    ]
]

<IPython.core.display.Javascript object>

In [425]:
(
    df_sorted_topo_models_cols.groupby("Model")[["Classification"]]
    .value_counts(normalize=True)
    .to_frame()
    * 100
).round(2)

proportion
Model           Classification                
Conv1D          intermediate_model       100.0
Neural Networks intermediate_model        50.0
                dominant_model            25.0
                non_dominant_model        25.0

<IPython.core.display.Javascript object>

In [426]:
df_sorted_topo_models_cols.groupby("Model").describe().round(2).T

Model                  Conv1D  Neural Networks
Dominated_Count count     1.0             4.00
                mean      1.0             2.00
                std       NaN             1.83
                min       1.0             0.00
                25%       1.0             0.75
                50%       1.0             2.00
                75%       1.0             3.25
                max       1.0             4.00
Dominates_Count count     1.0             4.00
                mean      2.0             1.75
                std       NaN             1.71
                min       2.0             0.00
                25%       2.0             0.75
                50%       2.0             1.50
                75%       2.0             2.50
                max       2.0             4.00

<IPython.core.display.Javascript object>

In [427]:
# df_sorted_topo_models_cols

<IPython.core.display.Javascript object>

In [428]:
df_sorted_topo_models_cols.groupby("Model")["Classification"].describe()

,count,unique,top,freq
Model,,,,
Conv1D,1,1,intermediate_model,1
Neural Networks,4,3,intermediate_model,2


<IPython.core.display.Javascript object>

In [429]:
df_sorted_topo_models_cols.groupby("Model")[["Classification"]].value_counts()

Model            Classification    
Conv1D           intermediate_model    1
Neural Networks  intermediate_model    2
                 dominant_model        1
                 non_dominant_model    1
Name: count, dtype: int64

<IPython.core.display.Javascript object>

In [430]:
summary_stats = df_sorted_topo_models_cols.groupby("Model").agg(
    Total_Models=("Model", "count"),
    Mean_Dominance_Count=("Dominates_Count", "mean"),
    Median_Dominance_Count=("Dominates_Count", "median"),
    Total_Dominated=("Dominated_Count", "sum"),
    Total_Dominating=("Dominates_Count", "sum"),
)

total_models = summary_stats[
    "Total_Models"
].sum()  # Get the total number of models across all categories
summary_stats["Dominance Proportion"] = summary_stats["Total_Dominating"] / total_models

<IPython.core.display.Javascript object>

In [431]:
summary_stats.round(2)

,Total_Models,Mean_Dominance_Count,Median_Dominance_Count,Total_Dominated,Total_Dominating,Dominance Proportion
Model,,,,,,
Conv1D,1,2.00,2.0,1,2,0.4
Neural Networks,4,1.75,1.5,8,7,1.4


<IPython.core.display.Javascript object>

In [432]:
df_sorted_topo_models_grouped.T

RMSE_mean                                                    \
                    count      mean       std       min       25%       50%   
Model                                                                         
Conv1D                1.0  1.887169       NaN  1.887169  1.887169  1.887169   
Neural Networks       4.0  4.230833  4.029891  1.555784  2.250549  2.567643   

                                     RMSE_std            ... Dominated_Count  \
                      75%        max    count      mean  ...             75%   
Model                                                    ...                   
Conv1D           1.887169   1.887169      1.0  0.370187  ...            1.00   
Neural Networks  4.547927  10.232263      4.0  2.904083  ...            3.25   

                     Dominates_Count                                            
                 max           count  mean       std  min   25%  50%  75%  max  
Model                                                                           
Conv1D           1.0             1.0  2.00       NaN  2.0  2.00  2.0  2.0  2.0  
Neural Networks  4.0             4.0  1.75  1.707825  0.0  0.75  1.5  2.5  4.0  

[2 rows x 88 columns]

<IPython.core.display.Javascript object>

### SCPM Analysis

In [433]:
cols = [
    # "Model",
    # "Model_bkp",
    "Model_bkp_2",
    "RMSE_mean",
    "RMSE_std",
    "MAE_mean",
    "MAE_std",
    "MAPE_mean",
    "MAPE_std",
    "R2_mean",
    "R2_std",
    "SCPM",
    "Dominated_Count",
    "Dominates_Count",
    "Classification",
    "Plant",
]

<IPython.core.display.Javascript object>

#### SCPM Analysis by Project

In [434]:
df_sorted_scpm = df_sorted_topo.sort_values(by="SCPM").copy()

<IPython.core.display.Javascript object>

In [435]:
# (df_sorted_topo[cols].reset_index() == df_sorted_scpm[cols].reset_index()).all(axis=1)
df_sorted_scpm["topo_scmp_order_eq"] = df_sorted_topo.index == df_sorted_scpm.index

<IPython.core.display.Javascript object>

In [436]:
df_sorted_scpm[cols + ["topo_scmp_order_eq"]].round(2).head(5)

,Model_bkp_2,RMSE_mean,RMSE_std,MAE_mean,MAE_std,MAPE_mean,MAPE_std,R2_mean,R2_std,SCPM,Dominated_Count,Dominates_Count,Classification,Plant,topo_scmp_order_eq
0,MLP,1.56,0.22,1.10,0.24,2.55,0.47,0.93,0.02,-4.34,0,4,dominant_model,c,True
1,Conv1D_14.0,1.89,0.37,1.43,0.26,3.37,0.66,0.89,0.05,-1.60,1,2,intermediate_model,c,True
3,LSTM_1.0,2.48,1.20,1.44,0.37,3.27,0.73,0.78,0.26,-1.48,1,2,intermediate_model,c,True
2,Transformer_1.0,2.65,1.49,1.72,0.91,3.91,1.87,0.73,0.32,0.72,3,1,intermediate_model,c,True
4,BidirectionalLSTM_1.0,10.23,8.70,1.88,0.68,4.27,1.62,-4.54,6.86,6.69,4,0,non_dominant_model,c,True


<IPython.core.display.Javascript object>